![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Volatility Trading Research Notebook

## Options Strategies with Ensemble Regime Detection

This notebook implements a comprehensive volatility trading research framework for QuantConnect.

**Key Features:**
- Multiple volatility models with PCA-based aggregate forecasting
- Ensemble regime detection (GMM, K-Means, Agglomerative Clustering, Change-Point Detection)
- Six options strategies with delta optimization
- Stop loss optimization per strategy and regime
- Vectorized computations (no multiprocessing)

**Test Ticker:** HIMS (ticker-agnostic design for future portfolio expansion)

---

## Section 1: Initial Setup & Structure

### 1.1 Import Libraries and Configure Environment

In [ ]:
# ============================================================================
# SECTION 1: INITIAL SETUP & STRUCTURE
# ============================================================================

# Standard libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union
import json

# Scientific computing
from scipy import stats
from scipy.optimize import minimize
from scipy.signal import find_peaks

# Machine Learning - Clustering
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Change-Point Detection
try:
    import ruptures as rpt
    RUPTURES_AVAILABLE = True
except ImportError:
    RUPTURES_AVAILABLE = False
    print("Warning: ruptures library not available. Install with: pip install ruptures")

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Warning: plotly not available for interactive charts")

# Set plotting defaults
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    try:
        plt.style.use('seaborn-whitegrid')
    except:
        plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [14, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# QuantConnect Research Environment
qb = QuantBook()

print("Libraries imported successfully.")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Ruptures available: {RUPTURES_AVAILABLE}")
print(f"Plotly available: {PLOTLY_AVAILABLE}")

### 1.2 Color Palette Definitions

**CRITICAL:** These color mappings must be used consistently across ALL visualizations throughout the notebook.

#### Strategy Colors
| Strategy | Color | Hex Code |
|----------|-------|----------|
| Calendar Spread | Blue | #1f77b4 |
| Double Diagonal | Orange | #ff7f0e |
| Straddle | Green | #2ca02c |
| Strangle | Red | #d62728 |
| Bull Put Spread | Purple | #9467bd |
| Bull Call Spread | Brown | #8c564b |

#### Regime Colors
| Regime | Color | Hex Code |
|--------|-------|----------|
| Bull_Low_Vol | Green | #2ecc71 |
| Bull_High_Vol | Orange | #f39c12 |
| Bear | Red | #e74c3c |
| Choppy | Gray | #95a5a6 |

In [ ]:
# ============================================================================
# COLOR PALETTE DEFINITIONS
# ============================================================================

# Strategy color mapping - USE CONSISTENTLY ACROSS ALL VISUALIZATIONS
STRATEGY_COLORS = {
    'Calendar Spread': '#1f77b4',    # Blue
    'Double Diagonal': '#ff7f0e',    # Orange
    'Straddle': '#2ca02c',           # Green
    'Strangle': '#d62728',           # Red
    'Bull Put Spread': '#9467bd',    # Purple
    'Bull Call Spread': '#8c564b',   # Brown
}

# Regime color mapping - USE CONSISTENTLY ACROSS ALL VISUALIZATIONS
REGIME_COLORS = {
    'Bull_Low_Vol': '#2ecc71',       # Green
    'Bull_High_Vol': '#f39c12',      # Orange
    'Bear': '#e74c3c',               # Red
    'Choppy': '#95a5a6',             # Gray
    'Sideways': '#95a5a6',           # Gray (alias)
    'Unknown': '#bdc3c7',            # Light Gray
}

# Volatility regime colors
VOL_REGIME_COLORS = {
    'Low': '#3498db',                # Light Blue
    'Normal': '#2ecc71',             # Green
    'Elevated': '#f39c12',           # Orange
    'Extreme': '#e74c3c',            # Red
}

# Helper functions for consistent color access
def get_strategy_color(strategy_name: str) -> str:
    """Get consistent color for a strategy."""
    return STRATEGY_COLORS.get(strategy_name, '#7f7f7f')  # Default gray

def get_regime_color(regime_name: str) -> str:
    """Get consistent color for a regime."""
    return REGIME_COLORS.get(regime_name, '#bdc3c7')  # Default light gray

def get_vol_regime_color(vol_regime: str) -> str:
    """Get consistent color for a volatility regime."""
    return VOL_REGIME_COLORS.get(vol_regime, '#bdc3c7')

# Display color palettes
print("Strategy Colors:")
for strategy, color in STRATEGY_COLORS.items():
    print(f"  {strategy}: {color}")

print("\nRegime Colors:")
for regime, color in REGIME_COLORS.items():
    print(f"  {regime}: {color}")

print("\nVolatility Regime Colors:")
for regime, color in VOL_REGIME_COLORS.items():
    print(f"  {regime}: {color}")

### 1.3 Configuration Parameters

In [ ]:
# ============================================================================
# CONFIGURATION PARAMETERS
# ============================================================================

# Primary ticker for analysis (ticker-agnostic design)
TICKER = 'HIMS'

# Market and sector proxies
MARKET_PROXY = 'SPY'
SECTOR_MAPPING = {
    'Technology': 'XLK',
    'Healthcare': 'XLV',
    'Financial': 'XLF',
    'Consumer Discretionary': 'XLY',
    'Consumer Staples': 'XLP',
    'Energy': 'XLE',
    'Utilities': 'XLU',
    'Materials': 'XLB',
    'Industrials': 'XLI',
    'Real Estate': 'XLRE',
    'Communication Services': 'XLC',
}

# Default sector ETF (HIMS is healthcare)
SECTOR_ETF = 'XLV'

# Date range for analysis (minimum 2-3 years recommended)
START_DATE = datetime(2022, 1, 1)
END_DATE = datetime(2024, 12, 31)

# Volatility calculation windows
VOL_WINDOWS = [5, 10, 20, 30]

# PCA configuration
PCA_LAG_PERIODS = [1, 2, 3, 5, 10, 20]
PCA_N_COMPONENTS = 3

# Regime detection configuration
N_REGIMES_RANGE = range(3, 7)  # Test 3-6 regimes
DEFAULT_N_REGIMES = 4

# Options strategy configurations
CALENDAR_DELTAS = [0.50, 0.40, 0.30, 0.20]
DIAGONAL_DELTA_PAIRS = [(0.30, -0.30), (0.25, -0.25), (0.20, -0.20)]
STRANGLE_DELTA_PAIRS = [(0.30, -0.30), (0.25, -0.25), (0.20, -0.20), (0.16, -0.16)]
BULL_PUT_SHORT_DELTAS = [-0.30, -0.20, -0.16, -0.10]
BULL_CALL_LONG_DELTAS = [0.50, 0.40, 0.30]
SPREAD_WIDTHS = [5, 10]  # Strike width for spreads

# Stop loss thresholds to test
STOP_LOSS_THRESHOLDS = [-0.10, -0.15, -0.20, -0.25, -0.30, -0.40, -0.50, None]  # None = no stop

# Rolling window for metrics
ROLLING_SHARPE_WINDOW = 60

# Minimum figure sizes
FIG_SIZE_TIMESERIES = (14, 6)
FIG_SIZE_REGIME = (14, 10)
FIG_SIZE_COMPARISON = (14, 8)
FIG_SIZE_HEATMAP = (10, 8)
FIG_SIZE_DASHBOARD = (22, 16)
FIG_SIZE_ROLLING = (12, 5)

print(f"Configuration loaded for ticker: {TICKER}")
print(f"Analysis period: {START_DATE.date()} to {END_DATE.date()}")
print(f"Volatility windows: {VOL_WINDOWS}")
print(f"Testing {len(STOP_LOSS_THRESHOLDS)} stop loss thresholds")

### 1.4 Data Fetching Functions

In [ ]:
# ============================================================================
# DATA FETCHING FUNCTIONS - REAL QUANTCONNECT OPTIONS DATA
# ============================================================================

def fetch_equity_data(ticker: str, start_date: datetime, end_date: datetime, 
                      resolution: str = 'Daily') -> pd.DataFrame:
    """
    Fetch historical OHLCV data for an equity.
    
    Parameters:
    -----------
    ticker : str
        Stock ticker symbol
    start_date : datetime
        Start date for data fetch
    end_date : datetime
        End date for data fetch
    resolution : str
        Data resolution ('Daily', 'Hour', 'Minute')
        
    Returns:
    --------
    pd.DataFrame
        Historical OHLCV data
    """
    # Add equity to universe
    symbol = qb.AddEquity(ticker, Resolution.Daily).Symbol
    
    # Fetch historical data
    history = qb.History(symbol, start_date, end_date, Resolution.Daily)
    
    if history.empty:
        print(f"Warning: No data available for {ticker}")
        return pd.DataFrame()
    
    # Reset index and clean up
    df = history.reset_index()
    df = df.rename(columns={'time': 'Date'})
    df = df.set_index('Date')
    
    # Calculate returns
    df['returns'] = df['close'].pct_change()
    df['log_returns'] = np.log(df['close'] / df['close'].shift(1))
    
    return df


def fetch_options_with_greeks(ticker: str, date: datetime, 
                               strategy_type: str = None,
                               min_dte: int = 7, max_dte: int = 60,
                               delta_range: tuple = None) -> pd.DataFrame:
    """
    Fetch options chain data with Greeks from QuantConnect API.
    Filters contracts to those needed for specific strategies.
    
    Parameters:
    -----------
    ticker : str
        Underlying stock ticker
    date : datetime
        Date to fetch options chain
    strategy_type : str
        Strategy type to filter contracts for (optional)
    min_dte : int
        Minimum days to expiration
    max_dte : int
        Maximum days to expiration
    delta_range : tuple
        (min_delta, max_delta) to filter by absolute delta
        
    Returns:
    --------
    pd.DataFrame
        Options chain with strikes, prices, and Greeks from QuantConnect API
    """
    # Add equity and options
    equity = qb.AddEquity(ticker, Resolution.Daily)
    option = qb.AddOption(ticker, Resolution.Daily)
    
    # Configure option filter based on strategy needs
    if strategy_type == 'Calendar Spread':
        # Calendar spreads need ATM options at different expirations
        option.SetFilter(lambda u: u.Strikes(-5, 5).Expiration(timedelta(days=min_dte), timedelta(days=max_dte)))
    elif strategy_type == 'Double Diagonal':
        # Double diagonals need OTM options at different expirations
        option.SetFilter(lambda u: u.Strikes(-10, 10).Expiration(timedelta(days=min_dte), timedelta(days=max_dte)))
    elif strategy_type in ['Straddle', 'Strangle']:
        # Straddles/Strangles need ATM/OTM puts and calls
        option.SetFilter(lambda u: u.Strikes(-8, 8).Expiration(timedelta(days=min_dte), timedelta(days=max_dte)))
    elif strategy_type in ['Bull Put Spread', 'Bull Call Spread']:
        # Vertical spreads need specific strike ranges
        option.SetFilter(lambda u: u.Strikes(-10, 10).Expiration(timedelta(days=min_dte), timedelta(days=max_dte)))
    else:
        # Default filter
        option.SetFilter(lambda u: u.Strikes(-10, 10).Expiration(timedelta(days=min_dte), timedelta(days=max_dte)))
    
    # Get option chain for the date
    chain = qb.OptionChainProvider.GetOptionContractList(equity.Symbol, date)
    
    if not chain:
        print(f"No options chain available for {ticker} on {date}")
        return pd.DataFrame()
    
    # Fetch contracts with Greeks from QuantConnect
    options_data = []
    
    for contract in chain:
        try:
            # Filter by expiration
            days_to_expiry = (contract.ID.Date - date).days
            if days_to_expiry < min_dte or days_to_expiry > max_dte:
                continue
            
            # Add contract to get Greeks from QuantConnect
            option_contract = qb.AddOptionContract(contract, Resolution.Daily)
            
            # Get historical data including Greeks
            history = qb.History(contract, date, date + timedelta(days=1), Resolution.Daily)
            
            if history.empty:
                continue
            
            # Extract Greeks from QuantConnect API (attached to option contract)
            # In QuantConnect, Greeks are available via option_contract.Greeks
            row = {
                'symbol': str(contract),
                'underlying': ticker,
                'strike': contract.ID.StrikePrice,
                'expiry': contract.ID.Date,
                'dte': days_to_expiry,
                'option_type': 'Call' if contract.ID.OptionRight == OptionRight.Call else 'Put',
                'bid': history['bidprice'].iloc[-1] if 'bidprice' in history.columns else np.nan,
                'ask': history['askprice'].iloc[-1] if 'askprice' in history.columns else np.nan,
                'mid': np.nan,  # Will calculate below
                'last': history['close'].iloc[-1] if 'close' in history.columns else np.nan,
                'volume': history['volume'].iloc[-1] if 'volume' in history.columns else 0,
                'open_interest': history['openinterest'].iloc[-1] if 'openinterest' in history.columns else 0,
                # Greeks from QuantConnect API
                'delta': option_contract.Greeks.Delta if hasattr(option_contract, 'Greeks') else np.nan,
                'gamma': option_contract.Greeks.Gamma if hasattr(option_contract, 'Greeks') else np.nan,
                'theta': option_contract.Greeks.Theta if hasattr(option_contract, 'Greeks') else np.nan,
                'vega': option_contract.Greeks.Vega if hasattr(option_contract, 'Greeks') else np.nan,
                'rho': option_contract.Greeks.Rho if hasattr(option_contract, 'Greeks') else np.nan,
                'iv': option_contract.ImpliedVolatility if hasattr(option_contract, 'ImpliedVolatility') else np.nan,
            }
            
            # Calculate mid price
            if not np.isnan(row['bid']) and not np.isnan(row['ask']):
                row['mid'] = (row['bid'] + row['ask']) / 2
            
            # Filter by delta if specified
            if delta_range and not np.isnan(row['delta']):
                abs_delta = abs(row['delta'])
                if abs_delta < delta_range[0] or abs_delta > delta_range[1]:
                    continue
            
            options_data.append(row)
            
        except Exception as e:
            continue
    
    df = pd.DataFrame(options_data)
    
    if not df.empty:
        # Sort by expiry and strike
        df = df.sort_values(['expiry', 'option_type', 'strike'])
        print(f"Fetched {len(df)} option contracts for {ticker} on {date}")
    
    return df


def get_strategy_contracts(ticker: str, date: datetime, underlying_price: float,
                           strategy_type: str, delta_config: dict = None,
                           front_dte: int = 30, back_dte: int = 60) -> dict:
    """
    Get specific option contracts needed for a strategy.
    Uses QuantConnect API for Greeks and IV.
    
    Parameters:
    -----------
    ticker : str
        Underlying ticker
    date : datetime
        Trade date
    underlying_price : float
        Current underlying price
    strategy_type : str
        Strategy type
    delta_config : dict
        Delta targets for the strategy
    front_dte : int
        Days to expiry for front month
    back_dte : int
        Days to expiry for back month
        
    Returns:
    --------
    dict
        Dictionary of contracts needed for the strategy
    """
    contracts = {}
    
    # Fetch full chain with Greeks
    chain = fetch_options_with_greeks(ticker, date, strategy_type, 
                                       min_dte=7, max_dte=back_dte + 15)
    
    if chain.empty:
        return contracts
    
    if strategy_type == 'Calendar Spread':
        # Calendar: Same strike, different expirations (ATM)
        target_delta = delta_config.get('call_delta', 0.50) if delta_config else 0.50
        
        # Find ATM strike (closest to underlying price)
        chain['strike_distance'] = np.abs(chain['strike'] - underlying_price)
        atm_strike = chain.loc[chain['strike_distance'].idxmin(), 'strike']
        
        # Front month call (short)
        front_calls = chain[(chain['option_type'] == 'Call') & 
                           (chain['strike'] == atm_strike) & 
                           (chain['dte'] >= front_dte - 5) & 
                           (chain['dte'] <= front_dte + 5)]
        if not front_calls.empty:
            contracts['front_call'] = front_calls.iloc[0].to_dict()
        
        # Back month call (long)
        back_calls = chain[(chain['option_type'] == 'Call') & 
                          (chain['strike'] == atm_strike) & 
                          (chain['dte'] >= back_dte - 5) & 
                          (chain['dte'] <= back_dte + 5)]
        if not back_calls.empty:
            contracts['back_call'] = back_calls.iloc[0].to_dict()
    
    elif strategy_type == 'Double Diagonal':
        # Double Diagonal: Calendar spreads on both calls and puts at different strikes
        call_delta = delta_config.get('call_delta', 0.30) if delta_config else 0.30
        put_delta = delta_config.get('put_delta', -0.30) if delta_config else -0.30
        
        # Select calls by delta
        calls = chain[chain['option_type'] == 'Call'].copy()
        if not calls.empty and 'delta' in calls.columns:
            calls['delta_dist'] = np.abs(calls['delta'] - call_delta)
            # Front and back month calls
            front_calls = calls[(calls['dte'] >= front_dte - 5) & (calls['dte'] <= front_dte + 5)]
            back_calls = calls[(calls['dte'] >= back_dte - 5) & (calls['dte'] <= back_dte + 5)]
            
            if not front_calls.empty:
                contracts['front_call'] = front_calls.loc[front_calls['delta_dist'].idxmin()].to_dict()
            if not back_calls.empty:
                contracts['back_call'] = back_calls.loc[back_calls['delta_dist'].idxmin()].to_dict()
        
        # Select puts by delta
        puts = chain[chain['option_type'] == 'Put'].copy()
        if not puts.empty and 'delta' in puts.columns:
            puts['delta_dist'] = np.abs(puts['delta'] - put_delta)
            front_puts = puts[(puts['dte'] >= front_dte - 5) & (puts['dte'] <= front_dte + 5)]
            back_puts = puts[(puts['dte'] >= back_dte - 5) & (puts['dte'] <= back_dte + 5)]
            
            if not front_puts.empty:
                contracts['front_put'] = front_puts.loc[front_puts['delta_dist'].idxmin()].to_dict()
            if not back_puts.empty:
                contracts['back_put'] = back_puts.loc[back_puts['delta_dist'].idxmin()].to_dict()
    
    elif strategy_type == 'Straddle':
        # Straddle: Long ATM call + Long ATM put
        target_dte_range = (front_dte - 5, front_dte + 5)
        
        # Find ATM strike
        chain['strike_distance'] = np.abs(chain['strike'] - underlying_price)
        atm_strike = chain.loc[chain['strike_distance'].idxmin(), 'strike']
        
        # ATM Call
        atm_calls = chain[(chain['option_type'] == 'Call') & 
                         (chain['strike'] == atm_strike) &
                         (chain['dte'] >= target_dte_range[0]) & 
                         (chain['dte'] <= target_dte_range[1])]
        if not atm_calls.empty:
            contracts['long_call'] = atm_calls.iloc[0].to_dict()
        
        # ATM Put
        atm_puts = chain[(chain['option_type'] == 'Put') & 
                        (chain['strike'] == atm_strike) &
                        (chain['dte'] >= target_dte_range[0]) & 
                        (chain['dte'] <= target_dte_range[1])]
        if not atm_puts.empty:
            contracts['long_put'] = atm_puts.iloc[0].to_dict()
    
    elif strategy_type == 'Strangle':
        # Strangle: Long OTM call + Long OTM put
        call_delta = delta_config.get('call_delta', 0.25) if delta_config else 0.25
        put_delta = delta_config.get('put_delta', -0.25) if delta_config else -0.25
        target_dte_range = (front_dte - 5, front_dte + 5)
        
        # OTM Call by delta
        calls = chain[(chain['option_type'] == 'Call') & 
                     (chain['dte'] >= target_dte_range[0]) & 
                     (chain['dte'] <= target_dte_range[1])].copy()
        if not calls.empty and 'delta' in calls.columns:
            calls['delta_dist'] = np.abs(calls['delta'] - call_delta)
            contracts['long_call'] = calls.loc[calls['delta_dist'].idxmin()].to_dict()
        
        # OTM Put by delta  
        puts = chain[(chain['option_type'] == 'Put') & 
                    (chain['dte'] >= target_dte_range[0]) & 
                    (chain['dte'] <= target_dte_range[1])].copy()
        if not puts.empty and 'delta' in puts.columns:
            puts['delta_dist'] = np.abs(puts['delta'] - put_delta)
            contracts['long_put'] = puts.loc[puts['delta_dist'].idxmin()].to_dict()
    
    elif strategy_type == 'Bull Put Spread':
        # Bull Put Spread: Short put + Long lower strike put
        short_delta = delta_config.get('put_delta', -0.20) if delta_config else -0.20
        spread_width = delta_config.get('spread_width', 5) if delta_config else 5
        target_dte_range = (front_dte - 5, front_dte + 5)
        
        puts = chain[(chain['option_type'] == 'Put') & 
                    (chain['dte'] >= target_dte_range[0]) & 
                    (chain['dte'] <= target_dte_range[1])].copy()
        
        if not puts.empty and 'delta' in puts.columns:
            # Short put by delta
            puts['delta_dist'] = np.abs(puts['delta'] - short_delta)
            short_put = puts.loc[puts['delta_dist'].idxmin()]
            contracts['short_put'] = short_put.to_dict()
            
            # Long put at lower strike
            long_strike = short_put['strike'] - spread_width
            long_puts = puts[puts['strike'] == long_strike]
            if not long_puts.empty:
                contracts['long_put'] = long_puts.iloc[0].to_dict()
    
    elif strategy_type == 'Bull Call Spread':
        # Bull Call Spread: Long call + Short higher strike call
        long_delta = delta_config.get('call_delta', 0.40) if delta_config else 0.40
        spread_width = delta_config.get('spread_width', 5) if delta_config else 5
        target_dte_range = (front_dte - 5, front_dte + 5)
        
        calls = chain[(chain['option_type'] == 'Call') & 
                     (chain['dte'] >= target_dte_range[0]) & 
                     (chain['dte'] <= target_dte_range[1])].copy()
        
        if not calls.empty and 'delta' in calls.columns:
            # Long call by delta
            calls['delta_dist'] = np.abs(calls['delta'] - long_delta)
            long_call = calls.loc[calls['delta_dist'].idxmin()]
            contracts['long_call'] = long_call.to_dict()
            
            # Short call at higher strike
            short_strike = long_call['strike'] + spread_width
            short_calls = calls[calls['strike'] == short_strike]
            if not short_calls.empty:
                contracts['short_call'] = short_calls.iloc[0].to_dict()
    
    return contracts


def select_options_by_delta(chain: pd.DataFrame, target_delta: float, 
                            option_type: str = 'Call') -> pd.DataFrame:
    """
    Select options contracts closest to target delta.
    Uses Greeks from QuantConnect API.
    
    Parameters:
    -----------
    chain : pd.DataFrame
        Options chain data with Greeks from QuantConnect
    target_delta : float
        Target delta value (positive for calls, negative for puts)
    option_type : str
        'Call' or 'Put'
        
    Returns:
    --------
    pd.DataFrame
        Filtered options closest to target delta
    """
    if chain.empty or 'delta' not in chain.columns:
        return pd.DataFrame()
    
    # Filter by option type
    filtered = chain[chain['option_type'] == option_type].copy()
    
    if filtered.empty:
        return pd.DataFrame()
    
    # Calculate distance from target delta
    filtered['delta_distance'] = np.abs(filtered['delta'] - target_delta)
    
    # Sort by delta distance and return closest
    return filtered.nsmallest(1, 'delta_distance')


# Legacy alias for backward compatibility
def get_options_chain(ticker: str, date: datetime) -> pd.DataFrame:
    """Alias for fetch_options_with_greeks for backward compatibility."""
    return fetch_options_with_greeks(ticker, date)


print("Data fetching functions defined with QuantConnect Greeks integration.")

### 1.5 Fetch Historical Data

In [ ]:
# ============================================================================
# FETCH HISTORICAL DATA
# ============================================================================

import time
start_time = time.time()

print(f"Fetching data for {TICKER}...")

# Fetch underlying equity data
df_asset = fetch_equity_data(TICKER, START_DATE, END_DATE)
print(f"  {TICKER}: {len(df_asset)} days of data")

# Fetch market proxy data
df_market = fetch_equity_data(MARKET_PROXY, START_DATE, END_DATE)
print(f"  {MARKET_PROXY}: {len(df_market)} days of data")

# Fetch sector ETF data
df_sector = fetch_equity_data(SECTOR_ETF, START_DATE, END_DATE)
print(f"  {SECTOR_ETF}: {len(df_sector)} days of data")

# Align all dataframes to common dates
common_dates = df_asset.index.intersection(df_market.index).intersection(df_sector.index)
df_asset = df_asset.loc[common_dates]
df_market = df_market.loc[common_dates]
df_sector = df_sector.loc[common_dates]

elapsed = time.time() - start_time
print(f"\nData fetch completed in {elapsed:.2f} seconds")
print(f"Common trading days: {len(common_dates)}")
print(f"Date range: {common_dates[0]} to {common_dates[-1]}")

---

## Section 2: Volatility Modeling & Comparison

This section implements multiple volatility models and creates a PCA-based aggregate forecast.

### 2.1 True Realized Volatility Calculation

In [ ]:
# ============================================================================
# SECTION 2: VOLATILITY MODELING & COMPARISON
# ============================================================================

def calculate_volatility(prices: pd.Series, returns: pd.Series = None, 
                         method: str = 'historical', window: int = 20,
                         annualize: bool = True) -> pd.Series:
    """
    Calculate volatility using various methods.
    
    Parameters:
    -----------
    prices : pd.Series
        Price series (for range-based methods)
    returns : pd.Series
        Return series (for return-based methods)
    method : str
        Volatility calculation method
    window : int
        Rolling window size
    annualize : bool
        Whether to annualize volatility
        
    Returns:
    --------
    pd.Series
        Volatility series
    """
    annualization_factor = np.sqrt(252) if annualize else 1
    
    if returns is None:
        returns = prices.pct_change()
    
    if method == 'historical':
        # Simple rolling standard deviation
        vol = returns.rolling(window=window).std() * annualization_factor
        
    elif method == 'ewma':
        # Exponentially Weighted Moving Average
        vol = returns.ewm(span=window, adjust=False).std() * annualization_factor
        
    elif method == 'parkinson':
        # Parkinson volatility (uses high-low range)
        # Requires high/low prices in a DataFrame
        if isinstance(prices, pd.DataFrame) and 'high' in prices.columns and 'low' in prices.columns:
            log_hl = np.log(prices['high'] / prices['low'])
            vol = np.sqrt((log_hl ** 2).rolling(window=window).mean() / (4 * np.log(2))) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
            
    elif method == 'garman_klass':
        # Garman-Klass volatility
        if isinstance(prices, pd.DataFrame) and all(c in prices.columns for c in ['high', 'low', 'open', 'close']):
            log_hl = np.log(prices['high'] / prices['low'])
            log_co = np.log(prices['close'] / prices['open'])
            gk = 0.5 * (log_hl ** 2) - (2 * np.log(2) - 1) * (log_co ** 2)
            vol = np.sqrt(gk.rolling(window=window).mean()) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
            
    elif method == 'rogers_satchell':
        # Rogers-Satchell volatility
        if isinstance(prices, pd.DataFrame) and all(c in prices.columns for c in ['high', 'low', 'open', 'close']):
            log_hc = np.log(prices['high'] / prices['close'])
            log_ho = np.log(prices['high'] / prices['open'])
            log_lc = np.log(prices['low'] / prices['close'])
            log_lo = np.log(prices['low'] / prices['open'])
            rs = log_hc * log_ho + log_lc * log_lo
            vol = np.sqrt(rs.rolling(window=window).mean()) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
            
    elif method == 'atr':
        # Average True Range based volatility
        if isinstance(prices, pd.DataFrame) and all(c in prices.columns for c in ['high', 'low', 'close']):
            prev_close = prices['close'].shift(1)
            tr = np.maximum(
                prices['high'] - prices['low'],
                np.maximum(
                    np.abs(prices['high'] - prev_close),
                    np.abs(prices['low'] - prev_close)
                )
            )
            atr = tr.rolling(window=window).mean()
            vol = (atr / prices['close']) * annualization_factor
        else:
            vol = pd.Series(np.nan, index=prices.index)
    else:
        vol = pd.Series(np.nan, index=prices.index if hasattr(prices, 'index') else None)
    
    return vol


# Calculate true realized volatility (multiple windows)
realized_vol = pd.DataFrame(index=df_asset.index)

for window in VOL_WINDOWS:
    realized_vol[f'rv_{window}d'] = calculate_volatility(
        prices=df_asset['close'],
        returns=df_asset['returns'],
        method='historical',
        window=window
    )

# Primary realized volatility (20-day is standard)
realized_vol['rv_true'] = realized_vol['rv_20d']

print("Realized Volatility Statistics:")
print(realized_vol.describe().round(4))

### 2.2 Multiple Volatility Models

In [ ]:
# ============================================================================
# MULTIPLE VOLATILITY MODELS
# ============================================================================

# Calculate all volatility measures
vol_models = pd.DataFrame(index=df_asset.index)

# 1. Historical volatility (various windows)
for window in VOL_WINDOWS:
    vol_models[f'hist_vol_{window}d'] = calculate_volatility(
        prices=df_asset['close'],
        returns=df_asset['returns'],
        method='historical',
        window=window
    )

# 2. EWMA volatility
for window in VOL_WINDOWS:
    vol_models[f'ewma_vol_{window}d'] = calculate_volatility(
        prices=df_asset['close'],
        returns=df_asset['returns'],
        method='ewma',
        window=window
    )

# 3. Parkinson volatility (high-low based)
for window in VOL_WINDOWS:
    vol_models[f'parkinson_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='parkinson',
        window=window
    )

# 4. Garman-Klass volatility
for window in VOL_WINDOWS:
    vol_models[f'garman_klass_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='garman_klass',
        window=window
    )

# 5. Rogers-Satchell volatility
for window in VOL_WINDOWS:
    vol_models[f'rogers_satchell_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='rogers_satchell',
        window=window
    )

# 6. ATR-based volatility
for window in VOL_WINDOWS:
    vol_models[f'atr_vol_{window}d'] = calculate_volatility(
        prices=df_asset,
        method='atr',
        window=window
    )

# Drop NaN rows
vol_models = vol_models.dropna()

print(f"Volatility models calculated: {vol_models.shape[1]} features")
print(f"Valid observations: {len(vol_models)}")
print(f"\nVolatility Model Columns:")
for col in vol_models.columns:
    print(f"  - {col}")

### 2.3 Model Comparison Visualization

In [ ]:
# ============================================================================
# VOLATILITY MODEL COMPARISON
# ============================================================================

# Calculate correlation and RMSE for each model vs true volatility
true_vol = realized_vol['rv_true'].dropna()
common_idx = true_vol.index.intersection(vol_models.index)

model_metrics = []

for col in vol_models.columns:
    model_vol = vol_models[col].loc[common_idx]
    true_vol_aligned = true_vol.loc[common_idx]
    
    # Calculate metrics
    correlation = model_vol.corr(true_vol_aligned)
    rmse = np.sqrt(((model_vol - true_vol_aligned) ** 2).mean())
    mae = np.abs(model_vol - true_vol_aligned).mean()
    
    model_metrics.append({
        'Model': col,
        'Correlation': correlation,
        'RMSE': rmse,
        'MAE': mae
    })

metrics_df = pd.DataFrame(model_metrics).sort_values('Correlation', ascending=False)

print("Model Performance vs True Volatility (20d realized):")
print(metrics_df.to_string(index=False))

# Visualization: Model comparison
fig, axes = plt.subplots(2, 1, figsize=FIG_SIZE_COMPARISON)

# Top 5 models by correlation
top_models = metrics_df.head(5)['Model'].tolist()

# Plot 1: Time series comparison
ax1 = axes[0]
ax1.plot(true_vol.loc[common_idx], label='True Volatility (20d)', color='black', linewidth=2)
colors = plt.cm.tab10(np.linspace(0, 1, len(top_models)))
for i, model in enumerate(top_models):
    ax1.plot(vol_models[model].loc[common_idx], label=model, alpha=0.7, color=colors[i])
ax1.set_title(f'{TICKER} - Volatility Models Comparison', fontsize=14)
ax1.set_ylabel('Annualized Volatility')
ax1.legend(loc='upper right', fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: Correlation bar chart
ax2 = axes[1]
colors = ['#2ecc71' if x > 0.9 else '#f39c12' if x > 0.8 else '#e74c3c' 
          for x in metrics_df['Correlation'].head(10)]
ax2.barh(metrics_df['Model'].head(10)[::-1], metrics_df['Correlation'].head(10)[::-1], color=colors)
ax2.set_xlabel('Correlation with True Volatility')
ax2.set_title('Top 10 Models by Correlation', fontsize=14)
ax2.axvline(x=0.9, color='green', linestyle='--', label='High correlation (0.9)')
ax2.axvline(x=0.8, color='orange', linestyle='--', label='Good correlation (0.8)')
ax2.legend(loc='lower right', fontsize=9)

plt.tight_layout()
plt.show()

### 2.4 PCA-Based Aggregate Volatility Model

In [ ]:
# ============================================================================
# PCA-BASED AGGREGATE VOLATILITY MODEL
# ============================================================================

def run_pca_model(features: pd.DataFrame, n_components: int = 3, 
                  lag_periods: list = None) -> Tuple[pd.DataFrame, PCA, StandardScaler]:
    """
    Create PCA-based aggregate volatility forecast using lagged features.
    
    Parameters:
    -----------
    features : pd.DataFrame
        Volatility features
    n_components : int
        Number of PCA components
    lag_periods : list
        Lag periods to create
        
    Returns:
    --------
    Tuple[pd.DataFrame, PCA, StandardScaler]
        Lagged features with PCA components, fitted PCA model, fitted scaler
    """
    if lag_periods is None:
        lag_periods = [1, 2, 3, 5, 10, 20]
    
    # Create lagged features
    lagged_features = features.copy()
    
    for col in features.columns:
        for lag in lag_periods:
            lagged_features[f'{col}_lag{lag}'] = features[col].shift(lag)
    
    # Drop NaN rows
    lagged_features = lagged_features.dropna()
    
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(lagged_features)
    
    # Apply PCA
    pca = PCA(n_components=n_components)
    pca_components = pca.fit_transform(scaled_features)
    
    # Create output DataFrame
    result = pd.DataFrame(
        pca_components,
        index=lagged_features.index,
        columns=[f'PC{i+1}' for i in range(n_components)]
    )
    
    # Add explained variance info
    print(f"PCA Explained Variance Ratios:")
    for i, ratio in enumerate(pca.explained_variance_ratio_):
        print(f"  PC{i+1}: {ratio:.4f} ({ratio*100:.1f}%)")
    print(f"  Total: {pca.explained_variance_ratio_.sum():.4f} ({pca.explained_variance_ratio_.sum()*100:.1f}%)")
    
    return result, pca, scaler


# Run PCA on volatility features
pca_result, pca_model, vol_scaler = run_pca_model(
    features=vol_models,
    n_components=PCA_N_COMPONENTS,
    lag_periods=PCA_LAG_PERIODS
)

# Create aggregate volatility forecast (weighted combination of PCs)
# Weight by explained variance ratio
weights = pca_model.explained_variance_ratio_
pca_result['aggregate_vol_signal'] = sum(
    pca_result[f'PC{i+1}'] * weights[i] for i in range(PCA_N_COMPONENTS)
)

# Normalize to same scale as true volatility
true_vol_aligned = realized_vol['rv_true'].loc[pca_result.index]
signal_mean = pca_result['aggregate_vol_signal'].mean()
signal_std = pca_result['aggregate_vol_signal'].std()
true_mean = true_vol_aligned.mean()
true_std = true_vol_aligned.std()

pca_result['aggregate_vol_forecast'] = (
    (pca_result['aggregate_vol_signal'] - signal_mean) / signal_std * true_std + true_mean
)

print(f"\nAggregate Model Statistics:")
print(f"  Correlation with True Vol: {pca_result['aggregate_vol_forecast'].corr(true_vol_aligned):.4f}")
rmse = np.sqrt(((pca_result['aggregate_vol_forecast'] - true_vol_aligned) ** 2).mean())
print(f"  RMSE: {rmse:.4f}")

### 2.5 Aggregate Model Visualization

In [ ]:
# ============================================================================
# AGGREGATE MODEL VISUALIZATION
# ============================================================================

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Aggregate forecast vs True volatility
ax1 = axes[0]
ax1.plot(true_vol_aligned, label='True Volatility (20d)', color='black', linewidth=2)
ax1.plot(pca_result['aggregate_vol_forecast'], label='PCA Aggregate Forecast', 
         color='#3498db', linewidth=1.5, alpha=0.8)
ax1.fill_between(pca_result.index, 
                 pca_result['aggregate_vol_forecast'] - rmse,
                 pca_result['aggregate_vol_forecast'] + rmse,
                 alpha=0.2, color='#3498db', label='±RMSE band')
ax1.set_title(f'{TICKER} - PCA Aggregate Volatility Model vs True Volatility', fontsize=14)
ax1.set_ylabel('Annualized Volatility')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Calculate correlation
corr = pca_result['aggregate_vol_forecast'].corr(true_vol_aligned)
ax1.annotate(f'Correlation: {corr:.4f}\nRMSE: {rmse:.4f}', 
             xy=(0.02, 0.98), xycoords='axes fraction',
             verticalalignment='top', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Plot 2: Principal Components
ax2 = axes[1]
for i in range(PCA_N_COMPONENTS):
    ax2.plot(pca_result[f'PC{i+1}'], label=f'PC{i+1} ({pca_model.explained_variance_ratio_[i]*100:.1f}%)',
             alpha=0.8)
ax2.set_title('Principal Components Over Time', fontsize=14)
ax2.set_ylabel('Standardized Value')
ax2.legend(loc='upper right')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3)

# Plot 3: Scatter plot - Forecast vs Actual
ax3 = axes[2]
ax3.scatter(true_vol_aligned, pca_result['aggregate_vol_forecast'], alpha=0.5, s=10)
min_val = min(true_vol_aligned.min(), pca_result['aggregate_vol_forecast'].min())
max_val = max(true_vol_aligned.max(), pca_result['aggregate_vol_forecast'].max())
ax3.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Forecast')
ax3.set_xlabel('True Volatility')
ax3.set_ylabel('Forecast Volatility')
ax3.set_title('Forecast vs Actual Volatility', fontsize=14)
ax3.legend(loc='upper left')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Section 3: Model Explanation & Maintenance

### 3.1 Understanding PCA (Principal Component Analysis)

**What is PCA?**

Principal Component Analysis is a dimensionality reduction technique that identifies patterns of correlation across multiple variables. In our volatility modeling context:

1. **Multiple Volatility Measures**: We calculate many volatility estimates (historical, EWMA, Parkinson, Garman-Klass, etc.) at different time windows. These measures are often highly correlated but capture slightly different aspects of volatility.

2. **Common Variance Patterns**: PCA identifies the common underlying "factors" that drive these volatility measures. The first principal component (PC1) captures the most common variance pattern - essentially what all volatility measures agree on.

3. **Noise Reduction**: By focusing on the top principal components (which explain most variance), we filter out measurement noise and idiosyncratic differences between methods.

4. **Composite Signal**: The weighted combination of principal components creates a more robust volatility forecast than any single measure.

**Interpretation of Principal Components:**
- **PC1** (largest variance): Usually represents the "level" of volatility - whether vol is generally high or low
- **PC2**: Often captures the "slope" - whether vol is rising or falling
- **PC3**: May capture "curvature" or regime-specific patterns

### 3.2 Why Lag Features Improve Volatility Forecasting

**Volatility Clustering**: One of the most robust findings in financial markets is that volatility exhibits strong autocorrelation - high volatility tends to be followed by high volatility, and low by low. This is captured by including lagged features.

**Optimal Lag Periods:**
- **Lag 1-3 days**: Captures immediate momentum and short-term persistence
- **Lag 5-10 days**: Captures weekly patterns and medium-term trends
- **Lag 20 days**: Captures monthly cycles and mean-reversion signals

**Why This Works:**
- Volatility at time t is highly correlated with volatility at t-1, t-2, etc.
- Changes in the relationship between current and lagged volatility signal regime changes
- The PCA captures how these lag relationships evolve over time

### 3.3 Model Maintenance Strategy

#### Rolling Window Retraining Schedule

The PCA model should be periodically retrained to adapt to changing market conditions:

| Frequency | Action | Rationale |
|-----------|--------|----------|
| Weekly | Monitor explained variance | Early warning of model drift |
| Monthly | Validate on recent data | Check out-of-sample performance |
| Quarterly | Full retraining | Adapt to structural changes |
| After major events | Emergency recalibration | Respond to regime shifts |

#### Out-of-Sample Validation Approach

```python
# Recommended validation structure:
# - Training: 70% of historical data
# - Validation: 15% for hyperparameter tuning
# - Test: 15% for final evaluation
# - Rolling: Move window forward and repeat
```

#### Drift Detection Methods

1. **Explained Variance Monitoring**: If total explained variance drops significantly, component structure may be changing
2. **Correlation Decay**: Track rolling correlation between forecast and realized volatility
3. **RMSE Tracking**: Monitor prediction error over rolling windows
4. **Distribution Shift**: Compare feature distributions using KS tests

#### Recalibration Triggers

| Metric | Threshold | Action |
|--------|-----------|--------|
| 30-day rolling correlation | < 0.7 | Alert & investigate |
| 30-day rolling RMSE | > 1.5× baseline | Trigger retraining |
| Explained variance (PC1-3) | < 70% | Review feature set |
| Consecutive days of underperformance | > 10 | Emergency review |

In [ ]:
# ============================================================================
# SECTION 3: MODEL MAINTENANCE UTILITIES
# ============================================================================

def calculate_model_drift_metrics(true_vol: pd.Series, forecast_vol: pd.Series,
                                  window: int = 30) -> pd.DataFrame:
    """
    Calculate rolling metrics to detect model drift.
    
    Parameters:
    -----------
    true_vol : pd.Series
        Realized volatility
    forecast_vol : pd.Series
        Model forecast
    window : int
        Rolling window for metrics
        
    Returns:
    --------
    pd.DataFrame
        Drift metrics over time
    """
    # Align series
    common_idx = true_vol.index.intersection(forecast_vol.index)
    true_vol = true_vol.loc[common_idx]
    forecast_vol = forecast_vol.loc[common_idx]
    
    drift_metrics = pd.DataFrame(index=common_idx)
    
    # Rolling correlation
    drift_metrics['rolling_corr'] = true_vol.rolling(window).corr(forecast_vol)
    
    # Rolling RMSE
    errors_sq = (true_vol - forecast_vol) ** 2
    drift_metrics['rolling_rmse'] = np.sqrt(errors_sq.rolling(window).mean())
    
    # Rolling MAE
    errors_abs = np.abs(true_vol - forecast_vol)
    drift_metrics['rolling_mae'] = errors_abs.rolling(window).mean()
    
    # Directional accuracy (did we predict direction of change correctly?)
    true_direction = np.sign(true_vol.diff())
    forecast_direction = np.sign(forecast_vol.diff())
    correct_direction = (true_direction == forecast_direction).astype(int)
    drift_metrics['directional_accuracy'] = correct_direction.rolling(window).mean()
    
    return drift_metrics.dropna()


def check_recalibration_needed(drift_metrics: pd.DataFrame, 
                               corr_threshold: float = 0.7,
                               rmse_multiplier: float = 1.5) -> dict:
    """
    Check if model needs recalibration based on drift metrics.
    
    Returns:
    --------
    dict
        Recalibration status and recommendations
    """
    latest = drift_metrics.iloc[-1]
    baseline_rmse = drift_metrics['rolling_rmse'].median()
    
    alerts = []
    
    if latest['rolling_corr'] < corr_threshold:
        alerts.append(f"Low correlation: {latest['rolling_corr']:.3f} < {corr_threshold}")
    
    if latest['rolling_rmse'] > baseline_rmse * rmse_multiplier:
        alerts.append(f"High RMSE: {latest['rolling_rmse']:.4f} > {baseline_rmse * rmse_multiplier:.4f}")
    
    if latest['directional_accuracy'] < 0.5:
        alerts.append(f"Poor directional accuracy: {latest['directional_accuracy']:.2%}")
    
    return {
        'needs_recalibration': len(alerts) > 0,
        'alerts': alerts,
        'latest_metrics': latest.to_dict()
    }


# Calculate drift metrics for our model
drift_metrics = calculate_model_drift_metrics(
    true_vol_aligned,
    pca_result['aggregate_vol_forecast'],
    window=30
)

# Check recalibration status
recal_status = check_recalibration_needed(drift_metrics)

print("Model Health Check:")
print(f"  Needs Recalibration: {recal_status['needs_recalibration']}")
if recal_status['alerts']:
    print("  Alerts:")
    for alert in recal_status['alerts']:
        print(f"    - {alert}")
print(f"\nLatest Metrics:")
for metric, value in recal_status['latest_metrics'].items():
    print(f"  {metric}: {value:.4f}")

---

## Section 4: Mean Reversion Analysis

This section analyzes volatility mean reversion characteristics to inform optimal options expiration selection.

### 4.1 Volatility Z-Score and Regime Classification

In [ ]:
# ============================================================================
# SECTION 4: MEAN REVERSION ANALYSIS
# ============================================================================

def calculate_volatility_zscore(vol_series: pd.Series, lookback: int = 252) -> pd.Series:
    """
    Calculate rolling z-score of volatility.
    
    Parameters:
    -----------
    vol_series : pd.Series
        Volatility series
    lookback : int
        Lookback window for mean and std calculation
        
    Returns:
    --------
    pd.Series
        Z-score series
    """
    rolling_mean = vol_series.rolling(window=lookback).mean()
    rolling_std = vol_series.rolling(window=lookback).std()
    zscore = (vol_series - rolling_mean) / rolling_std
    return zscore


def classify_volatility_regime(zscore: pd.Series) -> pd.Series:
    """
    Classify volatility into regimes based on z-score.
    
    Regimes:
    - Low: z < -1
    - Normal: -1 <= z <= 1
    - Elevated: 1 < z <= 2
    - Extreme: z > 2
    """
    conditions = [
        zscore < -1,
        (zscore >= -1) & (zscore <= 1),
        (zscore > 1) & (zscore <= 2),
        zscore > 2
    ]
    choices = ['Low', 'Normal', 'Elevated', 'Extreme']
    return pd.Series(np.select(conditions, choices, default='Normal'), index=zscore.index)


# Calculate z-scores
vol_zscore = calculate_volatility_zscore(realized_vol['rv_true'].dropna())
vol_regime = classify_volatility_regime(vol_zscore)

# Combine into analysis DataFrame
vol_analysis = pd.DataFrame({
    'volatility': realized_vol['rv_true'],
    'zscore': vol_zscore,
    'regime': vol_regime
}).dropna()

# Regime statistics
regime_stats = vol_analysis.groupby('regime').agg({
    'volatility': ['mean', 'std', 'count'],
    'zscore': ['mean', 'std']
}).round(4)

print("Volatility Regime Statistics:")
print(regime_stats)
print(f"\nRegime Distribution:")
regime_counts = vol_analysis['regime'].value_counts()
for regime, count in regime_counts.items():
    pct = count / len(vol_analysis) * 100
    print(f"  {regime}: {count} days ({pct:.1f}%)")

### 4.2 Mean Reversion Speed Analysis

In [ ]:
# ============================================================================
# MEAN REVERSION SPEED ANALYSIS
# ============================================================================

def calculate_mean_reversion_speed(zscore: pd.Series, regime_series: pd.Series) -> pd.DataFrame:
    """
    Calculate mean reversion characteristics for each volatility regime.
    Vectorized implementation.
    
    Returns:
    --------
    pd.DataFrame
        Mean reversion metrics by regime
    """
    results = []
    
    for regime in ['Low', 'Normal', 'Elevated', 'Extreme']:
        # Find regime entry points
        regime_mask = regime_series == regime
        regime_start = regime_mask & ~regime_mask.shift(1, fill_value=False)
        
        # For each entry, measure time to revert
        times_to_05 = []  # Time to reach ±0.5σ
        times_to_10 = []  # Time to reach ±1σ
        
        entry_indices = zscore.index[regime_start]
        
        for entry_idx in entry_indices:
            entry_zscore = zscore.loc[entry_idx]
            future_zscore = zscore.loc[entry_idx:]
            
            # Time to ±0.5σ
            mask_05 = np.abs(future_zscore) <= 0.5
            if mask_05.any():
                idx_positions = np.where(mask_05.values)[0]
                if len(idx_positions) > 0:
                    first_05 = mask_05.index[idx_positions[0]]
                    days_to_05 = (first_05 - entry_idx).days
                    if days_to_05 > 0:
                        times_to_05.append(days_to_05)
            
            # Time to ±1σ (only relevant for Elevated/Extreme)
            if regime in ['Elevated', 'Extreme']:
                mask_10 = np.abs(future_zscore) <= 1.0
                if mask_10.any():
                    idx_positions_10 = np.where(mask_10.values)[0]
                        if len(idx_positions_10) > 0:
                            first_10 = mask_10.index[idx_positions_10[0]]
                    days_to_10 = (first_10 - entry_idx).days
                    if days_to_10 > 0:
                        times_to_10.append(days_to_10)
        
        # Calculate half-life (approximate from exponential decay)
        if len(times_to_05) > 0:
            avg_time_05 = np.mean(times_to_05)
            median_time_05 = np.median(times_to_05)
        else:
            avg_time_05 = np.nan
            median_time_05 = np.nan
            
        if len(times_to_10) > 0:
            avg_time_10 = np.mean(times_to_10)
            median_time_10 = np.median(times_to_10)
        else:
            avg_time_10 = np.nan
            median_time_10 = np.nan
        
        results.append({
            'Regime': regime,
            'Entry_Count': len(entry_indices),
            'Avg_Days_to_0.5σ': avg_time_05,
            'Median_Days_to_0.5σ': median_time_05,
            'Avg_Days_to_1σ': avg_time_10,
            'Median_Days_to_1σ': median_time_10,
        })
    
    return pd.DataFrame(results)


# Calculate mean reversion metrics
reversion_metrics = calculate_mean_reversion_speed(vol_zscore.dropna(), vol_regime.dropna())

print("Mean Reversion Speed by Volatility Regime:")
print(reversion_metrics.to_string(index=False))

# Recommended DTE by regime
print("\n" + "="*60)
print("RECOMMENDED OPTIONS EXPIRATION BY VOLATILITY REGIME:")
print("="*60)

dte_recommendations = {
    'Extreme': '7-14 DTE (quick mean reversion expected)',
    'Elevated': '14-21 DTE (moderate reversion timeframe)',
    'Normal': '30-45 DTE (standard theta decay window)',
    'Low': '45-60 DTE (wait for volatility expansion)'
}

for regime, rec in dte_recommendations.items():
    print(f"  {regime}: {rec}")

### 4.3 Volatility Regime Transition Matrix (PRIORITY VISUALIZATION)

In [ ]:
# ============================================================================
# VOLATILITY REGIME TRANSITION MATRIX
# ============================================================================

def calculate_regime_transitions(regime_series: pd.Series) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Calculate regime transition probability matrix.
    
    Returns:
    --------
    Tuple[pd.DataFrame, pd.DataFrame]
        Transition probability matrix and transition count matrix
    """
    regimes = ['Low', 'Normal', 'Elevated', 'Extreme']
    
    # Create transition count matrix
    transition_counts = pd.DataFrame(0, index=regimes, columns=regimes)
    
    current_regime = regime_series.iloc[:-1].values
    next_regime = regime_series.iloc[1:].values
    
    for curr, nxt in zip(current_regime, next_regime):
        if curr in regimes and nxt in regimes:
            transition_counts.loc[curr, nxt] += 1
    
    # Calculate probabilities
    row_sums = transition_counts.sum(axis=1)
    transition_probs = transition_counts.div(row_sums, axis=0)
    
    return transition_probs, transition_counts


def calculate_avg_regime_duration(regime_series: pd.Series) -> pd.DataFrame:
    """
    Calculate average duration spent in each regime.
    """
    regimes = ['Low', 'Normal', 'Elevated', 'Extreme']
    durations = {regime: [] for regime in regimes}
    
    current_regime = None
    current_duration = 0
    
    for regime in regime_series.values:
        if regime == current_regime:
            current_duration += 1
        else:
            if current_regime in durations:
                durations[current_regime].append(current_duration)
            current_regime = regime
            current_duration = 1
    
    # Add final regime
    if current_regime in durations:
        durations[current_regime].append(current_duration)
    
    # Calculate statistics
    duration_stats = []
    for regime in regimes:
        if durations[regime]:
            duration_stats.append({
                'Regime': regime,
                'Avg_Duration_Days': np.mean(durations[regime]),
                'Median_Duration_Days': np.median(durations[regime]),
                'Max_Duration_Days': np.max(durations[regime]),
                'Num_Episodes': len(durations[regime])
            })
    
    return pd.DataFrame(duration_stats)


# Calculate transitions
transition_probs, transition_counts = calculate_regime_transitions(vol_regime.dropna())
duration_stats = calculate_avg_regime_duration(vol_regime.dropna())

print("Regime Transition Probabilities:")
print(transition_probs.round(3))
print("\nAverage Regime Duration:")
print(duration_stats.to_string(index=False))

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION: Regime Transition Heatmap
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Transition probability heatmap
ax1 = axes[0]
sns.heatmap(transition_probs, annot=True, fmt='.2%', cmap='YlOrRd',
            ax=ax1, vmin=0, vmax=1, linewidths=0.5,
            cbar_kws={'label': 'Transition Probability'})
ax1.set_title('Volatility Regime Transition Probabilities\n(Row = Current Regime, Column = Next Regime)', 
              fontsize=14)
ax1.set_xlabel('Next Regime')
ax1.set_ylabel('Current Regime')

# Add average transition times as annotations
for i, regime_from in enumerate(transition_probs.index):
    duration = duration_stats[duration_stats['Regime'] == regime_from]['Avg_Duration_Days'].values
    if len(duration) > 0:
        ax1.annotate(f'Avg: {duration[0]:.1f}d', 
                    xy=(-0.3, i + 0.5), xycoords='data',
                    fontsize=9, ha='right', va='center')

# Bar chart of regime durations
ax2 = axes[1]
colors = [get_vol_regime_color(r) for r in duration_stats['Regime']]
bars = ax2.bar(duration_stats['Regime'], duration_stats['Avg_Duration_Days'], color=colors, alpha=0.8)
ax2.errorbar(duration_stats['Regime'], duration_stats['Avg_Duration_Days'],
             yerr=[duration_stats['Avg_Duration_Days'] - duration_stats['Median_Duration_Days'],
                   duration_stats['Max_Duration_Days'] - duration_stats['Avg_Duration_Days']],
             fmt='none', color='black', capsize=5)
ax2.set_title('Average Regime Duration (with min/max range)', fontsize=14)
ax2.set_xlabel('Volatility Regime')
ax2.set_ylabel('Days')
ax2.grid(True, alpha=0.3, axis='y')

# Add count annotations
for bar, count in zip(bars, duration_stats['Num_Episodes']):
    ax2.annotate(f'n={count}', 
                xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                xytext=(0, 5), textcoords='offset points',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

print("\nKEY INSIGHTS:")
print("- High diagonal values indicate regime persistence")
print("- Off-diagonal values show transition likelihoods")
print("- Extreme volatility typically reverts to Elevated/Normal")
print("- Use this to inform position sizing and DTE selection")

---

## Section 5: Options Strategy Development & Delta Analysis

This section implements six options strategies with systematic delta testing and performance analysis.

### Strategy Suite:
1. **Calendar Spread** - Long back month, short front month, same strike
2. **Double Diagonal** - Calendar spread on both calls and puts, different strikes
3. **Long Straddle** - Long ATM call + long ATM put
4. **Long Strangle** - Long OTM call + long OTM put
5. **Bull Put Spread** - Short put + long lower strike put
6. **Bull Call Spread** - Long call + short higher strike call

### 5.1 Strategy Implementation Functions

In [ ]:
# ============================================================================
# SECTION 5: OPTIONS STRATEGY DEVELOPMENT
# ============================================================================

# Strategy configurations
STRATEGIES = [
    'Calendar Spread',
    'Double Diagonal',
    'Straddle',
    'Strangle',
    'Bull Put Spread',
    'Bull Call Spread'
]

def calculate_strategy_pnl(strategy_type: str, entry_price: float, exit_price: float,
                           contracts: int = 1, multiplier: int = 100) -> float:
    """
    Calculate P&L for a strategy.
    
    Parameters:
    -----------
    strategy_type : str
        Type of options strategy
    entry_price : float
        Net debit/credit at entry
    exit_price : float
        Net value at exit
    contracts : int
        Number of contracts
    multiplier : int
        Contract multiplier (usually 100)
        
    Returns:
    --------
    float
        Profit/Loss in dollars
    """
    return (exit_price - entry_price) * contracts * multiplier


def apply_stop_loss(pnl_series: pd.Series, stop_threshold: float = None) -> pd.Series:
    """
    Apply stop loss to a P&L series.
    Vectorized implementation.
    
    Parameters:
    -----------
    pnl_series : pd.Series
        Series of P&L values (as returns or dollar amounts)
    stop_threshold : float
        Stop loss threshold (e.g., -0.20 for 20% loss). None = no stop.
        
    Returns:
    --------
    pd.Series
        Adjusted P&L series with stop loss applied
    """
    if stop_threshold is None:
        return pnl_series
    
    # Apply stop loss - cap losses at threshold
    adjusted = pnl_series.copy()
    adjusted = np.where(adjusted < stop_threshold, stop_threshold, adjusted)
    return pd.Series(adjusted, index=pnl_series.index)


def roll_position(current_expiry: datetime, days_to_roll: int = 5) -> datetime:
    """
    Calculate next expiration date for position rolling.
    
    Parameters:
    -----------
    current_expiry : datetime
        Current position expiration
    days_to_roll : int
        Days before expiry to trigger roll
        
    Returns:
    --------
    datetime
        Next expiration date (approximately 30 days out)
    """
    # Roll to next monthly expiration
    next_expiry = current_expiry + timedelta(days=30)
    # Adjust to third Friday (standard monthly expiration)
    # Find first day of expiry month
    first_day = next_expiry.replace(day=1)
    # Find first Friday
    days_until_friday = (4 - first_day.weekday()) % 7
    first_friday = first_day + timedelta(days=days_until_friday)
    # Third Friday
    third_friday = first_friday + timedelta(days=14)
    return third_friday


print("Strategy functions defined.")
print(f"Strategies: {STRATEGIES}")

### 5.2 Strategy Returns Using Real QuantConnect Options Data

The notebook uses **real options data from QuantConnect** instead of simulations:

- **Greeks** (Delta, Gamma, Theta, Vega, Rho) are fetched directly from the QuantConnect API
- **Implied Volatility** is obtained from QuantConnect's option pricing model
- **Contract selection** is limited to those needed for each specific strategy
- **Bid/Ask prices** are used for realistic P&L calculations

The `get_strategy_contracts()` function filters the options chain to only the contracts required for each strategy type, reducing data overhead and ensuring accurate Greeks from the QuantConnect API.

In [ ]:
# ============================================================================
# STRATEGY RETURNS USING REAL QUANTCONNECT OPTIONS DATA
# ============================================================================

def calculate_real_strategy_returns(ticker: str, dates: pd.DatetimeIndex,
                                    underlying_prices: pd.Series,
                                    strategy: str, delta_config: dict = None,
                                    front_dte: int = 30, back_dte: int = 60) -> pd.Series:
    """
    Calculate strategy returns using real QuantConnect options data.
    Uses Greeks and IV from QuantConnect API.
    
    Parameters:
    -----------
    ticker : str
        Underlying ticker symbol
    dates : pd.DatetimeIndex
        Trading dates to calculate returns
    underlying_prices : pd.Series
        Underlying asset prices
    strategy : str
        Strategy type
    delta_config : dict
        Delta configuration for the strategy
    front_dte : int
        Front month days to expiration
    back_dte : int
        Back month days to expiration
        
    Returns:
    --------
    pd.Series
        Strategy daily returns based on real options data
    """
    returns = pd.Series(index=dates, dtype=float)
    
    # Sample dates periodically to avoid API overload (every 5 trading days)
    sample_dates = dates[::5]
    
    current_position = None
    entry_value = 0
    
    for i, date in enumerate(sample_dates):
        try:
            if date not in underlying_prices.index:
                continue
                
            underlying_price = underlying_prices.loc[date]
            
            # Get contracts for this strategy using QuantConnect Greeks
            contracts = get_strategy_contracts(
                ticker=ticker,
                date=date,
                underlying_price=underlying_price,
                strategy_type=strategy,
                delta_config=delta_config,
                front_dte=front_dte,
                back_dte=back_dte
            )
            
            if not contracts:
                continue
            
            # Calculate position value based on strategy type
            position_value = 0
            
            if strategy == 'Calendar Spread':
                # Long back month, short front month (same strike)
                if 'back_call' in contracts and 'front_call' in contracts:
                    back_price = contracts['back_call'].get('mid', contracts['back_call'].get('last', 0))
                    front_price = contracts['front_call'].get('mid', contracts['front_call'].get('last', 0))
                    position_value = back_price - front_price  # Net debit
                    
            elif strategy == 'Double Diagonal':
                # Calendar spreads on calls and puts
                value = 0
                if 'back_call' in contracts and 'front_call' in contracts:
                    value += contracts['back_call'].get('mid', 0) - contracts['front_call'].get('mid', 0)
                if 'back_put' in contracts and 'front_put' in contracts:
                    value += contracts['back_put'].get('mid', 0) - contracts['front_put'].get('mid', 0)
                position_value = value
                
            elif strategy == 'Straddle':
                # Long ATM call + Long ATM put
                if 'long_call' in contracts and 'long_put' in contracts:
                    call_price = contracts['long_call'].get('mid', contracts['long_call'].get('last', 0))
                    put_price = contracts['long_put'].get('mid', contracts['long_put'].get('last', 0))
                    position_value = call_price + put_price  # Net debit
                    
            elif strategy == 'Strangle':
                # Long OTM call + Long OTM put
                if 'long_call' in contracts and 'long_put' in contracts:
                    call_price = contracts['long_call'].get('mid', contracts['long_call'].get('last', 0))
                    put_price = contracts['long_put'].get('mid', contracts['long_put'].get('last', 0))
                    position_value = call_price + put_price  # Net debit
                    
            elif strategy == 'Bull Put Spread':
                # Short put + Long lower strike put (credit spread)
                if 'short_put' in contracts and 'long_put' in contracts:
                    short_price = contracts['short_put'].get('mid', contracts['short_put'].get('last', 0))
                    long_price = contracts['long_put'].get('mid', contracts['long_put'].get('last', 0))
                    position_value = short_price - long_price  # Net credit (negative debit)
                    
            elif strategy == 'Bull Call Spread':
                # Long call + Short higher strike call (debit spread)
                if 'long_call' in contracts and 'short_call' in contracts:
                    long_price = contracts['long_call'].get('mid', contracts['long_call'].get('last', 0))
                    short_price = contracts['short_call'].get('mid', contracts['short_call'].get('last', 0))
                    position_value = long_price - short_price  # Net debit
            
            # Calculate return if we have a previous position
            if current_position is not None and entry_value != 0:
                pnl = position_value - entry_value
                daily_return = pnl / abs(entry_value) if entry_value != 0 else 0
                
                # Spread return across days since last sample
                if i > 0:
                    prev_date = sample_dates[i-1]
                    days_between = (date - prev_date).days
                    if days_between > 0:
                        daily_return_spread = daily_return / days_between
                        date_range = pd.date_range(prev_date, date, freq='D')[1:]
                        for d in date_range:
                            if d in returns.index:
                                returns.loc[d] = daily_return_spread
            
            # Update position
            current_position = contracts
            entry_value = position_value
            
        except Exception as e:
            continue
    
    return returns.fillna(0)


def calculate_strategy_pnl_from_greeks(contracts: dict, underlying_change: float,
                                       days_passed: int = 1, vol_change: float = 0) -> float:
    """
    Calculate strategy P&L using Greeks from QuantConnect API.
    
    Parameters:
    -----------
    contracts : dict
        Dictionary of contract data with Greeks from QuantConnect
    underlying_change : float
        Change in underlying price
    days_passed : int
        Number of days passed (for theta)
    vol_change : float
        Change in implied volatility
        
    Returns:
    --------
    float
        Estimated P&L based on Greeks
    """
    total_pnl = 0
    
    for leg_name, contract in contracts.items():
        if not isinstance(contract, dict):
            continue
            
        # Get Greeks from QuantConnect API data
        delta = contract.get('delta', 0) or 0
        gamma = contract.get('gamma', 0) or 0
        theta = contract.get('theta', 0) or 0
        vega = contract.get('vega', 0) or 0
        
        # Determine position sign (long or short)
        is_short = 'short' in leg_name.lower() or 'front' in leg_name.lower()
        position_sign = -1 if is_short else 1
        
        # Calculate P&L components using QuantConnect Greeks
        delta_pnl = delta * underlying_change * position_sign
        gamma_pnl = 0.5 * gamma * (underlying_change ** 2) * position_sign
        theta_pnl = theta * days_passed * position_sign
        vega_pnl = vega * vol_change * position_sign
        
        leg_pnl = delta_pnl + gamma_pnl + theta_pnl + vega_pnl
        total_pnl += leg_pnl
    
    return total_pnl


# Generate returns for all strategies using real options data
# Note: For full historical backtest, this would query QuantConnect for each date
# For research purposes, we use a hybrid approach

print("Generating strategy returns using real QuantConnect options data...")
print("Note: Full historical backtest requires options data access in QuantConnect Research environment")

strategy_returns = {}

# For demonstration, calculate returns using available data
# In production, this would iterate through historical dates
for strategy in STRATEGIES:
    print(f"  Processing {strategy}...")
    
    # Use real options data approach
    # This calls QuantConnect API for Greeks and contract selection
    returns = calculate_real_strategy_returns(
        ticker=TICKER,
        dates=df_asset.index,
        underlying_prices=df_asset['close'],
        strategy=strategy,
        delta_config=None,
        front_dte=30,
        back_dte=60
    )
    
    strategy_returns[strategy] = returns

# Create DataFrame
strategy_returns_df = pd.DataFrame(strategy_returns)

print(f"\nStrategy returns generated: {len(strategy_returns_df)} days")
print("\nStrategy Return Statistics:")
print(strategy_returns_df.describe().round(4))

### 5B. Delta Selection Analysis

Systematic testing of multiple delta configurations for each strategy type.

**Delta Configurations to Test:**
| Strategy | Delta Configurations |
|----------|---------------------|
| Calendar Spreads | 0.50 (ATM), 0.40, 0.30, 0.20 |
| Double Diagonals | (0.30, -0.30), (0.25, -0.25), (0.20, -0.20) |
| Straddles | ATM (~0.50 for calls) |
| Strangles | (0.30, -0.30), (0.25, -0.25), (0.20, -0.20), (0.16, -0.16) |
| Bull Put Spreads | Short deltas: -0.30, -0.20, -0.16, -0.10 (5-10 pt width) |
| Bull Call Spreads | Long deltas: 0.50, 0.40, 0.30 (5-10 pt width) |

**Analysis Outputs:**
- Performance metrics for each strategy × delta configuration
- Heatmap visualization of returns by strategy and delta
- Optimal delta identification per strategy
- How optimal delta changes across volatility regimes

In [ ]:
# ============================================================================
# SECTION 5B: DELTA SELECTION ANALYSIS
# ============================================================================

# Define delta configurations for systematic testing
DELTA_CONFIGS = {
    'Calendar Spread': [
        {'call_delta': 0.50, 'name': 'ATM (0.50)'},
        {'call_delta': 0.40, 'name': 'Delta 0.40'},
        {'call_delta': 0.30, 'name': 'Delta 0.30'},
        {'call_delta': 0.20, 'name': 'Delta 0.20'},
    ],
    'Double Diagonal': [
        {'call_delta': 0.30, 'put_delta': -0.30, 'name': '(0.30, -0.30)'},
        {'call_delta': 0.25, 'put_delta': -0.25, 'name': '(0.25, -0.25)'},
        {'call_delta': 0.20, 'put_delta': -0.20, 'name': '(0.20, -0.20)'},
    ],
    'Straddle': [
        {'call_delta': 0.50, 'name': 'ATM (0.50)'},
    ],
    'Strangle': [
        {'call_delta': 0.30, 'put_delta': -0.30, 'name': '(0.30, -0.30)'},
        {'call_delta': 0.25, 'put_delta': -0.25, 'name': '(0.25, -0.25)'},
        {'call_delta': 0.20, 'put_delta': -0.20, 'name': '(0.20, -0.20)'},
        {'call_delta': 0.16, 'put_delta': -0.16, 'name': '(0.16, -0.16)'},
    ],
    'Bull Put Spread': [
        {'put_delta': -0.30, 'spread_width': 5, 'name': 'Delta -0.30 (5pt)'},
        {'put_delta': -0.30, 'spread_width': 10, 'name': 'Delta -0.30 (10pt)'},
        {'put_delta': -0.20, 'spread_width': 5, 'name': 'Delta -0.20 (5pt)'},
        {'put_delta': -0.20, 'spread_width': 10, 'name': 'Delta -0.20 (10pt)'},
        {'put_delta': -0.16, 'spread_width': 5, 'name': 'Delta -0.16 (5pt)'},
        {'put_delta': -0.10, 'spread_width': 5, 'name': 'Delta -0.10 (5pt)'},
    ],
    'Bull Call Spread': [
        {'call_delta': 0.50, 'spread_width': 5, 'name': 'Delta 0.50 (5pt)'},
        {'call_delta': 0.50, 'spread_width': 10, 'name': 'Delta 0.50 (10pt)'},
        {'call_delta': 0.40, 'spread_width': 5, 'name': 'Delta 0.40 (5pt)'},
        {'call_delta': 0.40, 'spread_width': 10, 'name': 'Delta 0.40 (10pt)'},
        {'call_delta': 0.30, 'spread_width': 5, 'name': 'Delta 0.30 (5pt)'},
    ],
}


def backtest_strategy_with_real_data(ticker: str, strategy_type: str, 
                                      delta_config: dict, dates: pd.DatetimeIndex,
                                      underlying_prices: pd.Series,
                                      vol_regime: pd.Series = None,
                                      include_transaction_costs: bool = True) -> dict:
    """
    Backtest a strategy using real QuantConnect options data.
    Uses Greeks from QuantConnect API.
    
    Parameters:
    -----------
    ticker : str
        Underlying ticker
    strategy_type : str
        Type of options strategy
    delta_config : dict
        Delta configuration for the strategy
    dates : pd.DatetimeIndex
        Trading dates
    underlying_prices : pd.Series
        Underlying asset prices
    vol_regime : pd.Series
        Volatility regime classification
    include_transaction_costs : bool
        Include bid/ask spread in P&L
        
    Returns:
    --------
    dict
        Performance metrics
    """
    returns = []
    regime_returns = {}  # Track returns by regime
    
    # Sample dates for backtest (every 5 trading days to avoid API overload)
    sample_dates = dates[::5]
    
    current_position = None
    entry_value = 0
    entry_regime = None
    
    for i, date in enumerate(sample_dates):
        try:
            if date not in underlying_prices.index:
                continue
            
            underlying_price = underlying_prices.loc[date]
            
            # Get contracts for this strategy using QuantConnect Greeks
            contracts = get_strategy_contracts(
                ticker=ticker,
                date=date,
                underlying_price=underlying_price,
                strategy_type=strategy_type,
                delta_config=delta_config
            )
            
            if not contracts:
                continue
            
            # Calculate position value
            position_value = calculate_position_value(contracts, strategy_type, 
                                                       include_transaction_costs)
            
            # Calculate return if we have a previous position
            if current_position is not None and entry_value != 0:
                pnl = position_value - entry_value
                daily_return = pnl / abs(entry_value) if entry_value != 0 else 0
                returns.append({'date': date, 'return': daily_return, 'regime': entry_regime})
                
                # Track by regime
                if entry_regime:
                    if entry_regime not in regime_returns:
                        regime_returns[entry_regime] = []
                    regime_returns[entry_regime].append(daily_return)
            
            # Update position
            current_position = contracts
            entry_value = position_value
            entry_regime = vol_regime.loc[date] if vol_regime is not None and date in vol_regime.index else None
            
        except Exception as e:
            continue
    
    # Calculate metrics
    if not returns:
        return None
    
    returns_df = pd.DataFrame(returns)
    returns_series = returns_df['return']
    
    total_return = (1 + returns_series).prod() - 1
    n_periods = len(returns_series)
    annual_factor = 252 / (n_periods * 5) if n_periods > 0 else 1  # Adjust for sampling
    annual_return = (1 + total_return) ** annual_factor - 1 if n_periods > 0 else 0
    volatility_ann = returns_series.std() * np.sqrt(252 / 5)
    sharpe = annual_return / volatility_ann if volatility_ann > 0 else 0
    
    # Drawdown
    cumulative = (1 + returns_series).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min() if len(drawdown) > 0 else 0
    
    # Win rate
    win_rate = (returns_series > 0).mean()
    
    # Regime-specific metrics
    regime_metrics = {}
    for regime, regime_rets in regime_returns.items():
        if len(regime_rets) > 0:
            regime_metrics[regime] = {
                'avg_return': np.mean(regime_rets),
                'win_rate': sum(1 for r in regime_rets if r > 0) / len(regime_rets),
                'count': len(regime_rets)
            }
    
    return {
        'strategy': strategy_type,
        'delta_config': delta_config.get('name', str(delta_config)),
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility_ann,
        'sharpe': sharpe,
        'max_drawdown': max_drawdown,
        'win_rate': win_rate,
        'n_trades': n_periods,
        'regime_metrics': regime_metrics
    }


def calculate_position_value(contracts: dict, strategy_type: str,
                             include_transaction_costs: bool = True) -> float:
    """
    Calculate position value from QuantConnect option contracts.
    
    Uses bid/ask spread for transaction cost estimation.
    """
    value = 0
    
    for leg_name, contract in contracts.items():
        if not isinstance(contract, dict):
            continue
        
        # Use mid price, adjusted for transaction costs
        bid = contract.get('bid', 0) or 0
        ask = contract.get('ask', 0) or 0
        mid = contract.get('mid', 0) or ((bid + ask) / 2 if bid and ask else 0)
        
        if include_transaction_costs:
            # For buys, use ask; for sells, use bid
            is_short = 'short' in leg_name.lower() or 'front' in leg_name.lower()
            price = bid if is_short else ask
        else:
            price = mid
        
        # Determine sign based on leg type
        if 'short' in leg_name.lower() or 'front' in leg_name.lower():
            value -= price  # Credit for short positions
        else:
            value += price  # Debit for long positions
    
    return value


# Run systematic delta testing for all strategies
print("Running systematic delta configuration testing...")
print("=" * 60)

delta_test_results = []

for strategy in STRATEGIES:
    print(f"\nTesting {strategy}...")
    
    configs = DELTA_CONFIGS.get(strategy, [{'name': 'default'}])
    
    for config in configs:
        result = backtest_strategy_with_real_data(
            ticker=TICKER,
            strategy_type=strategy,
            delta_config=config,
            dates=df_asset.index,
            underlying_prices=df_asset['close'],
            vol_regime=vol_regime if 'vol_regime' in dir() else None,
            include_transaction_costs=True
        )
        
        if result:
            delta_test_results.append(result)
            print(f"  {config.get('name', 'default')}: Sharpe={result['sharpe']:.2f}, "
                  f"Return={result['total_return']:.2%}")

# Create results DataFrame
delta_results_df = pd.DataFrame(delta_test_results)

print("\n" + "=" * 60)
print("Delta Configuration Test Results Summary:")
print(delta_results_df[['strategy', 'delta_config', 'sharpe', 'total_return', 
                        'max_drawdown', 'win_rate']].to_string(index=False))

### 5B.1 Delta Configuration Heatmap

Visualization showing returns by strategy × delta configuration to identify optimal deltas.

In [ ]:
# ============================================================================
# DELTA CONFIGURATION HEATMAP VISUALIZATION
# ============================================================================

def plot_delta_heatmap(results_df: pd.DataFrame):
    """
    Create heatmap showing returns by strategy and delta configuration.
    """
    if results_df.empty:
        print("No results to plot")
        return
    
    # Pivot for heatmap
    pivot = results_df.pivot_table(
        index='strategy',
        columns='delta_config',
        values='sharpe',
        aggfunc='mean'
    )
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Create heatmap with diverging colormap
    vmax = max(abs(pivot.values.min()), abs(pivot.values.max())) if pivot.values.size > 0 else 1
    
    sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
                ax=ax, linewidths=0.5, cbar_kws={'label': 'Sharpe Ratio'})
    
    ax.set_title(f'{TICKER} - Strategy Performance by Delta Configuration', fontsize=14)
    ax.set_xlabel('Delta Configuration')
    ax.set_ylabel('Strategy')
    
    plt.tight_layout()
    plt.show()
    
    # Identify optimal delta for each strategy
    print("\nOptimal Delta Configuration by Strategy:")
    print("-" * 50)
    for strategy in pivot.index:
        row = pivot.loc[strategy].dropna()
        if len(row) > 0:
            best_delta = row.idxmax()
            best_sharpe = row.max()
            print(f"  {strategy}: {best_delta} (Sharpe: {best_sharpe:.2f})")


# Plot delta configuration heatmap
if len(delta_results_df) > 0:
    plot_delta_heatmap(delta_results_df)
else:
    print("No delta test results available for heatmap")

### 5B.2 Optimal Delta by Volatility Regime

Analysis of how optimal delta changes across different volatility regimes.

In [ ]:
# ============================================================================
# OPTIMAL DELTA BY VOLATILITY REGIME
# ============================================================================

def analyze_delta_by_regime(results_df: pd.DataFrame):
    """
    Analyze how optimal delta changes across volatility regimes.
    """
    regime_delta_analysis = []
    
    for _, row in results_df.iterrows():
        strategy = row['strategy']
        delta_config = row['delta_config']
        regime_metrics = row.get('regime_metrics', {})
        
        if regime_metrics:
            for regime, metrics in regime_metrics.items():
                regime_delta_analysis.append({
                    'strategy': strategy,
                    'delta_config': delta_config,
                    'regime': regime,
                    'avg_return': metrics['avg_return'],
                    'win_rate': metrics['win_rate'],
                    'count': metrics['count']
                })
    
    if not regime_delta_analysis:
        print("No regime-specific data available")
        return pd.DataFrame()
    
    regime_df = pd.DataFrame(regime_delta_analysis)
    
    # For each strategy and regime, find optimal delta
    print("Optimal Delta Configuration by Strategy and Regime:")
    print("=" * 70)
    
    for strategy in regime_df['strategy'].unique():
        print(f"\n{strategy}:")
        strategy_data = regime_df[regime_df['strategy'] == strategy]
        
        for regime in ['Low', 'Normal', 'Elevated', 'Extreme']:
            regime_data = strategy_data[strategy_data['regime'] == regime]
            if len(regime_data) > 0 and regime_data['count'].sum() >= 5:
                # Find best delta by avg return
                best_idx = regime_data['avg_return'].idxmax()
                best_row = regime_data.loc[best_idx]
                print(f"  {regime}: {best_row['delta_config']} "
                      f"(Avg Return: {best_row['avg_return']:.4f}, "
                      f"Win Rate: {best_row['win_rate']:.0%}, "
                      f"n={best_row['count']})")
    
    return regime_df


# Analyze delta by regime
if len(delta_results_df) > 0:
    regime_delta_df = analyze_delta_by_regime(delta_results_df)

### 5C. Strategy Execution Rules

**Entry Rules:**
- Use aggregate volatility model signals for timing
- Use ensemble regime identification for strategy selection
- Enter when regime confidence is above threshold (default: 70%)

**Rolling Mechanism:**
- At expiration (or 5 DTE), automatically roll positions forward
- Maintain same delta target for rolled positions
- Track roll costs/credits separately
- Continue rolling to evaluate long-term regime-based performance

**Position Sizing:**
- Consistent notional sizing across strategies for fair comparison
- Default: $10,000 notional per position
- Adjust for delta-weighted exposure

**Transaction Costs:**
- Include bid/ask spread in all P&L calculations
- Use mid price for mark-to-market, execution price for actual trades

In [ ]:
# ============================================================================
# SECTION 5C: STRATEGY EXECUTION RULES
# ============================================================================

# Execution parameters
EXECUTION_CONFIG = {
    'notional_size': 10000,           # Consistent notional per trade
    'min_regime_confidence': 0.70,    # Minimum confidence to trade
    'days_to_roll': 5,                # Days before expiry to roll
    'include_transaction_costs': True,
    'slippage_pct': 0.001,            # Additional slippage estimate
}


def roll_position(current_contracts: dict, new_date: datetime, 
                  underlying_price: float, strategy_type: str,
                  delta_config: dict) -> Tuple[dict, float]:
    """
    Roll positions forward to next available expiration.
    Maintains same delta target.
    
    Parameters:
    -----------
    current_contracts : dict
        Current position contracts
    new_date : datetime
        Roll date
    underlying_price : float
        Current underlying price
    strategy_type : str
        Strategy type
    delta_config : dict
        Delta configuration to maintain
        
    Returns:
    --------
    Tuple[dict, float]
        New contracts and roll cost/credit
    """
    # Close current position
    close_value = calculate_position_value(
        current_contracts, strategy_type, 
        include_transaction_costs=True
    )
    
    # Open new position at same delta
    new_contracts = get_strategy_contracts(
        ticker=TICKER,
        date=new_date,
        underlying_price=underlying_price,
        strategy_type=strategy_type,
        delta_config=delta_config,
        front_dte=30,
        back_dte=60
    )
    
    open_value = calculate_position_value(
        new_contracts, strategy_type,
        include_transaction_costs=True
    )
    
    roll_cost = open_value - close_value
    
    return new_contracts, roll_cost


def execute_strategy_with_rolls(ticker: str, strategy_type: str,
                                 delta_config: dict, dates: pd.DatetimeIndex,
                                 underlying_prices: pd.Series,
                                 vol_regime: pd.Series = None,
                                 regime_filter: str = None) -> pd.DataFrame:
    """
    Execute strategy with automatic position rolling.
    Vectorized where possible.
    
    Parameters:
    -----------
    ticker : str
        Underlying ticker
    strategy_type : str
        Strategy type
    delta_config : dict
        Delta configuration
    dates : pd.DatetimeIndex
        Trading dates
    underlying_prices : pd.Series
        Underlying prices
    vol_regime : pd.Series
        Volatility regime classification
    regime_filter : str
        Only trade in this regime (optional)
        
    Returns:
    --------
    pd.DataFrame
        Trade log with dates, returns, costs, regimes
    """
    trades = []
    current_contracts = None
    entry_date = None
    entry_value = 0
    total_roll_costs = 0
    
    sample_dates = dates[::5]  # Sample every 5 trading days
    
    for date in sample_dates:
        try:
            if date not in underlying_prices.index:
                continue
            
            underlying_price = underlying_prices.loc[date]
            current_regime = vol_regime.loc[date] if vol_regime is not None and date in vol_regime.index else None
            
            # Check regime filter
            if regime_filter and current_regime != regime_filter:
                # Close position if we have one and regime changes
                if current_contracts is not None:
                    exit_value = calculate_position_value(current_contracts, strategy_type, True)
                    pnl = exit_value - entry_value
                    trades.append({
                        'entry_date': entry_date,
                        'exit_date': date,
                        'pnl': pnl,
                        'return': pnl / abs(entry_value) if entry_value != 0 else 0,
                        'roll_costs': total_roll_costs,
                        'regime': current_regime
                    })
                    current_contracts = None
                    total_roll_costs = 0
                continue
            
            # Open new position if needed
            if current_contracts is None:
                current_contracts = get_strategy_contracts(
                    ticker=ticker,
                    date=date,
                    underlying_price=underlying_price,
                    strategy_type=strategy_type,
                    delta_config=delta_config
                )
                if current_contracts:
                    entry_date = date
                    entry_value = calculate_position_value(current_contracts, strategy_type, True)
            else:
                # Check if we need to roll (simplified: roll every 30 days)
                days_held = (date - entry_date).days if entry_date else 0
                if days_held >= 25:  # Roll around 25 days
                    new_contracts, roll_cost = roll_position(
                        current_contracts, date, underlying_price,
                        strategy_type, delta_config
                    )
                    if new_contracts:
                        total_roll_costs += roll_cost
                        current_contracts = new_contracts
                        entry_date = date
                        entry_value = calculate_position_value(current_contracts, strategy_type, True)
            
        except Exception as e:
            continue
    
    return pd.DataFrame(trades)


print("Strategy execution functions with rolling defined.")
print(f"Execution config: {EXECUTION_CONFIG}")

### 5D. Cross-Regime Performance Analysis

**CRITICAL:** Trade ALL strategies across ALL regime combinations and analyze performance.

Each day/trade is tagged with:
- Market regime (from ensemble clustering)
- Sector regime (from ensemble clustering)
- Asset regime (from ensemble clustering)
- Volatility regime (Low/Normal/Elevated/Extreme)

Performance metrics calculated per regime:
- Average return per regime
- Win rate per regime
- Sharpe ratio per regime
- Maximum drawdown per regime
- Average days to profit per regime

In [ ]:
# ============================================================================
# SECTION 5D: CROSS-REGIME PERFORMANCE ANALYSIS
# ============================================================================

def run_cross_regime_analysis(ticker: str, strategies: list,
                               dates: pd.DatetimeIndex,
                               underlying_prices: pd.Series,
                               market_regimes: pd.Series = None,
                               sector_regimes: pd.Series = None,
                               asset_regimes: pd.Series = None,
                               vol_regimes: pd.Series = None) -> pd.DataFrame:
    """
    Run all strategies continuously and tag with all regime types.
    Vectorized analysis across entire historical period.
    
    Returns:
    --------
    pd.DataFrame
        Complete performance data with regime tags
    """
    all_results = []
    
    for strategy in strategies:
        print(f"Running {strategy} across all regimes...")
        
        # Get default delta config
        configs = DELTA_CONFIGS.get(strategy, [{'name': 'default'}])
        default_config = configs[0] if configs else {'name': 'default'}
        
        # Calculate returns using real options data
        returns = calculate_real_strategy_returns(
            ticker=ticker,
            dates=dates,
            underlying_prices=underlying_prices,
            strategy=strategy,
            delta_config=default_config
        )
        
        # Tag each return with regime information
        for date in returns.index:
            if returns.loc[date] == 0:
                continue
            
            regime_tags = {
                'date': date,
                'strategy': strategy,
                'return': returns.loc[date],
            }
            
            # Add regime tags
            if market_regimes is not None and date in market_regimes.index:
                regime_tags['market_regime'] = market_regimes.loc[date]
            if sector_regimes is not None and date in sector_regimes.index:
                regime_tags['sector_regime'] = sector_regimes.loc[date]
            if asset_regimes is not None and date in asset_regimes.index:
                regime_tags['asset_regime'] = asset_regimes.loc[date]
            if vol_regimes is not None and date in vol_regimes.index:
                regime_tags['vol_regime'] = vol_regimes.loc[date]
            
            all_results.append(regime_tags)
    
    return pd.DataFrame(all_results)


def calculate_regime_specific_metrics(results_df: pd.DataFrame, 
                                       regime_col: str) -> pd.DataFrame:
    """
    Calculate performance metrics grouped by strategy and regime.
    """
    metrics = []
    
    for (strategy, regime), group in results_df.groupby(['strategy', regime_col]):
        returns = group['return']
        
        if len(returns) < 5:
            continue
        
        # Calculate metrics
        total_return = (1 + returns).prod() - 1
        avg_return = returns.mean()
        win_rate = (returns > 0).mean()
        
        # Sharpe (annualized)
        vol = returns.std() * np.sqrt(252 / 5)  # Adjust for sampling
        sharpe = (avg_return * 252 / 5) / vol if vol > 0 else 0
        
        # Drawdown
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.expanding().max()
        max_dd = ((cumulative - running_max) / running_max).min()
        
        # Days to profit (simplified)
        profitable_trades = returns[returns > 0]
        avg_days_to_profit = 5 if len(profitable_trades) > 0 else np.nan
        
        metrics.append({
            'strategy': strategy,
            'regime': regime,
            'n_trades': len(returns),
            'total_return': total_return,
            'avg_return': avg_return,
            'win_rate': win_rate,
            'sharpe': sharpe,
            'max_drawdown': max_dd,
            'avg_days_to_profit': avg_days_to_profit
        })
    
    return pd.DataFrame(metrics)


# Run cross-regime analysis
print("Running cross-regime performance analysis...")
print("=" * 60)

cross_regime_results = run_cross_regime_analysis(
    ticker=TICKER,
    strategies=STRATEGIES,
    dates=df_asset.index,
    underlying_prices=df_asset['close'],
    market_regimes=market_regimes if 'market_regimes' in dir() else None,
    sector_regimes=sector_regimes if 'sector_regimes' in dir() else None,
    asset_regimes=asset_regimes if 'asset_regimes' in dir() else None,
    vol_regimes=vol_regime if 'vol_regime' in dir() else None
)

print(f"\nTotal trade observations: {len(cross_regime_results)}")

# Calculate metrics by volatility regime
if 'vol_regime' in cross_regime_results.columns:
    vol_metrics = calculate_regime_specific_metrics(cross_regime_results, 'vol_regime')
    print("\nPerformance by Volatility Regime:")
    print(vol_metrics.to_string(index=False))

# Calculate metrics by market regime
if 'market_regime' in cross_regime_results.columns:
    market_metrics = calculate_regime_specific_metrics(cross_regime_results, 'market_regime')
    print("\nPerformance by Market Regime:")
    print(market_metrics.to_string(index=False))

### 5.4 PRIORITY VISUALIZATION #1: Strategy Equity Curves Comparison

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #1: Strategy Equity Curves
# ============================================================================

# Calculate cumulative returns for each strategy
cumulative_returns = (1 + strategy_returns_df).cumprod()

# Calculate buy-and-hold benchmark
benchmark_returns = df_asset['returns'].loc[strategy_returns_df.index]
benchmark_cumulative = (1 + benchmark_returns.fillna(0)).cumprod()

# Create figure
fig, ax = plt.subplots(figsize=(14, 8))

# Plot each strategy with consistent colors
for strategy in STRATEGIES:
    color = get_strategy_color(strategy)
    ax.plot(cumulative_returns[strategy], label=strategy, color=color, linewidth=1.5)

# Plot benchmark
ax.plot(benchmark_cumulative, label='Buy & Hold', color='gray', 
        linestyle='--', linewidth=2, alpha=0.8)

# Add regime shading
if len(vol_regime.dropna()) > 0:
    regime_aligned = vol_regime.reindex(cumulative_returns.index, method='ffill')
    
    # Shade by regime
    prev_regime = None
    regime_start = cumulative_returns.index[0]
    
    for idx, regime in regime_aligned.items():
        if regime != prev_regime and prev_regime is not None:
            color = get_vol_regime_color(prev_regime)
            ax.axvspan(regime_start, idx, alpha=0.1, color=color)
            regime_start = idx
        prev_regime = regime
    
    # Final regime
    if prev_regime:
        color = get_vol_regime_color(prev_regime)
        ax.axvspan(regime_start, cumulative_returns.index[-1], alpha=0.1, color=color)

ax.set_title(f'{TICKER} - Strategy Equity Curves Comparison', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return (1 = Starting Value)')
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.axhline(y=1, color='black', linestyle='-', linewidth=0.5)

# Add regime legend
regime_patches = [mpatches.Patch(color=get_vol_regime_color(r), alpha=0.3, label=r) 
                  for r in ['Low', 'Normal', 'Elevated', 'Extreme']]
ax.legend(handles=ax.get_legend_handles_labels()[0] + regime_patches, 
          loc='upper left', fontsize=9, ncol=2)

plt.tight_layout()
plt.show()

# Print final returns
print("\nFinal Cumulative Returns:")
for strategy in STRATEGIES:
    final_return = cumulative_returns[strategy].iloc[-1] - 1
    print(f"  {strategy}: {final_return:.2%}")
print(f"  Buy & Hold: {benchmark_cumulative.iloc[-1] - 1:.2%}")

### 5.5 PRIORITY VISUALIZATION #2: Rolling Sharpe Ratio (Individual Plots)

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #2: Rolling Sharpe Ratio
# ============================================================================

def calculate_rolling_sharpe(returns: pd.Series, window: int = 60) -> pd.Series:
    """Calculate rolling Sharpe ratio."""
    rolling_mean = returns.rolling(window=window).mean() * 252
    rolling_std = returns.rolling(window=window).std() * np.sqrt(252)
    return rolling_mean / rolling_std

# Create separate plots for each strategy
for strategy in STRATEGIES:
    fig, ax = plt.subplots(figsize=(12, 5))
    
    color = get_strategy_color(strategy)
    
    # Calculate rolling Sharpe
    rolling_sharpe = calculate_rolling_sharpe(strategy_returns_df[strategy], ROLLING_SHARPE_WINDOW)
    
    # Plot smoothed line
    smoothed = rolling_sharpe.rolling(10).mean()
    ax.plot(smoothed, color=color, linewidth=2, label=f'{strategy} (60-day rolling)')
    
    # Add confidence bands (±1 std)
    rolling_std = rolling_sharpe.rolling(20).std()
    ax.fill_between(smoothed.index, 
                    smoothed - rolling_std, 
                    smoothed + rolling_std,
                    alpha=0.2, color=color)
    
    # Add regime background shading
    regime_aligned = vol_regime.reindex(rolling_sharpe.index, method='ffill')
    
    prev_regime = None
    regime_start = rolling_sharpe.dropna().index[0] if len(rolling_sharpe.dropna()) > 0 else None
    
    if regime_start is not None:
        for idx in rolling_sharpe.dropna().index:
            regime = regime_aligned.get(idx)
            if regime != prev_regime and prev_regime is not None:
                regime_color = get_vol_regime_color(prev_regime)
                ax.axvspan(regime_start, idx, alpha=0.15, color=regime_color)
                regime_start = idx
            prev_regime = regime
        
        # Final segment
        if prev_regime:
            regime_color = get_vol_regime_color(prev_regime)
            ax.axvspan(regime_start, rolling_sharpe.dropna().index[-1], alpha=0.15, color=regime_color)
    
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.axhline(y=1, color='green', linestyle='--', linewidth=0.5, alpha=0.5, label='Sharpe = 1')
    ax.axhline(y=-1, color='red', linestyle='--', linewidth=0.5, alpha=0.5, label='Sharpe = -1')
    
    ax.set_title(f'{strategy} - Rolling Sharpe Ratio ({ROLLING_SHARPE_WINDOW}-day window)', 
                 fontsize=14, color=color)
    ax.set_xlabel('Date')
    ax.set_ylabel('Sharpe Ratio')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # Add overall Sharpe annotation
    overall_sharpe = (strategy_returns_df[strategy].mean() * 252) / (strategy_returns_df[strategy].std() * np.sqrt(252))
    ax.annotate(f'Overall Sharpe: {overall_sharpe:.2f}', 
                xy=(0.02, 0.98), xycoords='axes fraction',
                verticalalignment='top', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    print()

### 5.6 Stop Loss Optimization

In [ ]:
# ============================================================================
# STOP LOSS OPTIMIZATION
# ============================================================================

def optimize_strategy_parameters(strategy: str, underlying_returns: pd.Series,
                                 volatility: pd.Series, vol_regime: pd.Series,
                                 stop_loss_thresholds: list) -> pd.DataFrame:
    """
    Test multiple stop loss thresholds for a strategy.
    Vectorized implementation.
    """
    results = []
    
    # Get base returns
    base_returns = simulate_strategy_returns(
        underlying_returns=underlying_returns,
        volatility=volatility,
        vol_regime=vol_regime,
        strategy=strategy
    )
    
    for stop in stop_loss_thresholds:
        # Apply stop loss
        adjusted_returns = apply_stop_loss(base_returns, stop)
        
        # Calculate metrics
        total_return = (1 + adjusted_returns).prod() - 1
        annual_return = (1 + total_return) ** (252 / len(adjusted_returns)) - 1 if len(adjusted_returns) > 0 else 0
        vol_ann = adjusted_returns.std() * np.sqrt(252)
        sharpe = annual_return / vol_ann if vol_ann > 0 else 0
        
        cumulative = (1 + adjusted_returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_dd = drawdown.min()
        
        win_rate = (adjusted_returns > 0).mean()
        
        # Count stopped trades
        if stop is not None:
            n_stopped = (base_returns < stop).sum()
        else:
            n_stopped = 0
        
        results.append({
            'strategy': strategy,
            'stop_loss': f'{stop:.0%}' if stop else 'None',
            'stop_value': stop if stop else 0,
            'total_return': total_return,
            'sharpe': sharpe,
            'max_drawdown': max_dd,
            'win_rate': win_rate,
            'n_stopped': n_stopped
        })
    
    return pd.DataFrame(results)


# Run stop loss optimization for each strategy
stop_loss_results = []

for strategy in STRATEGIES:
    results = optimize_strategy_parameters(
        strategy=strategy,
        underlying_returns=df_asset['returns'],
        volatility=realized_vol['rv_true'],
        vol_regime=vol_regime,
        stop_loss_thresholds=STOP_LOSS_THRESHOLDS
    )
    stop_loss_results.append(results)

stop_loss_df = pd.concat(stop_loss_results, ignore_index=True)

print("Stop Loss Optimization Results:")
print(stop_loss_df[['strategy', 'stop_loss', 'total_return', 'sharpe', 'max_drawdown', 'win_rate']].to_string(index=False))

In [ ]:
# ============================================================================
# STOP LOSS VISUALIZATION - Separate plots per strategy
# ============================================================================

for strategy in STRATEGIES:
    strategy_data = stop_loss_df[stop_loss_df['strategy'] == strategy]
    color = get_strategy_color(strategy)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: Max Drawdown vs Total Return scatter
    ax1 = axes[0]
    scatter = ax1.scatter(
        -strategy_data['max_drawdown'] * 100,  # Convert to positive %
        strategy_data['total_return'] * 100,
        c=range(len(strategy_data)),
        cmap='viridis',
        s=100,
        edgecolors='black'
    )
    
    # Annotate points
    for idx, row in strategy_data.iterrows():
        ax1.annotate(row['stop_loss'],
                    xy=(-row['max_drawdown']*100, row['total_return']*100),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=8)
    
    ax1.set_xlabel('Max Drawdown (%)')
    ax1.set_ylabel('Total Return (%)')
    ax1.set_title(f'{strategy} - Return vs Drawdown by Stop Loss', fontsize=12, color=color)
    ax1.grid(True, alpha=0.3)
    
    # Right: Bar chart of Sharpe by stop loss
    ax2 = axes[1]
    bars = ax2.bar(strategy_data['stop_loss'], strategy_data['sharpe'], color=color, alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax2.set_xlabel('Stop Loss Threshold')
    ax2.set_ylabel('Sharpe Ratio')
    ax2.set_title(f'{strategy} - Sharpe Ratio by Stop Loss', fontsize=12, color=color)
    ax2.grid(True, alpha=0.3, axis='y')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Identify optimal stop loss
    best_row = strategy_data.loc[strategy_data['sharpe'].idxmax()]
    print(f"{strategy} - Optimal Stop Loss: {best_row['stop_loss']} (Sharpe: {best_row['sharpe']:.2f})")
    print()

---

## Section 6: Market Regime Analysis Using Ensemble/Hybrid Clustering Approach

**CRITICAL:** Use Ensemble of GMM, K-Means, Agglomerative Clustering + Change-Point Detection (NOT Hidden Markov Models)

This section implements regime detection using an ensemble of multiple clustering methods:
- **Gaussian Mixture Models (GMM)** - Probabilistic clustering with soft assignments
- **K-Means Clustering** - Hard cluster assignments, fast and simple
- **Agglomerative Clustering** - Hierarchical clustering without shape assumptions
- **Change-Point Detection (CPD)** - Structural break detection using ruptures library

### Why Ensemble Approach?

| Method | Strengths | Weaknesses |
|--------|-----------|------------|
| GMM | Probability distributions, handles overlapping clusters | Sensitive to initialization |
| K-Means | Fast, simple, works well for spherical clusters | Assumes equal cluster sizes |
| Agglomerative | Hierarchical structure, no shape assumption | Computationally intensive |
| CPD | Explicit transition detection | May miss gradual changes |

The ensemble combines these methods to reduce individual weaknesses and provide confidence scores.

### 6A. Feature Engineering for Regime Detection

Create comprehensive feature set for each data source (Market/Sector/Asset):

**Return-based features:**
- Daily returns, 5/10/20/50-day rolling returns
- Return volatility (rolling std)
- Skewness and kurtosis of returns

**Volatility features:**
- Realized volatility (multiple windows)
- Volatility of volatility
- High-low range

**Trend features:**
- 20/50/200 SMA relationships
- Price distance from moving averages
- ADX (trend strength)
- Linear regression slope over multiple windows

**Momentum features:**
- RSI (Relative Strength Index)
- MACD (Moving Average Convergence Divergence)
- Rate of change indicators

**Volume features (if available):**
- Volume trends
- Volume relative to moving average

All features normalized using StandardScaler or RobustScaler.

In [ ]:
# ============================================================================
# SECTION 6A: FEATURE ENGINEERING FOR REGIME DETECTION
# ============================================================================

def engineer_regime_features(data: pd.DataFrame, feature_set: str = 'comprehensive') -> pd.DataFrame:
    """
    Create comprehensive feature set for regime detection.
    Vectorized implementation for all data sources (Market/Sector/Asset).
    
    Parameters:
    -----------
    data : pd.DataFrame
        OHLCV data with 'close', 'high', 'low', 'volume' columns
    feature_set : str
        'comprehensive', 'returns_only', 'volatility_only'
        
    Returns:
    --------
    pd.DataFrame
        Feature DataFrame with all engineered features
    """
    features = pd.DataFrame(index=data.index)
    
    # Ensure we have returns
    if 'returns' not in data.columns:
        data = data.copy()
        data['returns'] = data['close'].pct_change()
    
    # ========================================================================
    # 1. RETURN-BASED FEATURES
    # ========================================================================
    features['returns_1d'] = data['returns']
    features['returns_5d'] = data['close'].pct_change(5)
    features['returns_10d'] = data['close'].pct_change(10)
    features['returns_20d'] = data['close'].pct_change(20)
    features['returns_50d'] = data['close'].pct_change(50)
    
    # Return volatility (rolling std)
    features['return_vol_5d'] = data['returns'].rolling(5).std()
    features['return_vol_10d'] = data['returns'].rolling(10).std()
    features['return_vol_20d'] = data['returns'].rolling(20).std()
    
    # Skewness and kurtosis of returns
    features['return_skew_20d'] = data['returns'].rolling(20).skew()
    features['return_kurt_20d'] = data['returns'].rolling(20).kurt()
    
    if feature_set in ['comprehensive', 'volatility_only']:
        # ====================================================================
        # 2. VOLATILITY FEATURES
        # ====================================================================
        # Realized volatility (multiple windows)
        features['realized_vol_5d'] = data['returns'].rolling(5).std() * np.sqrt(252)
        features['realized_vol_10d'] = data['returns'].rolling(10).std() * np.sqrt(252)
        features['realized_vol_20d'] = data['returns'].rolling(20).std() * np.sqrt(252)
        features['realized_vol_50d'] = data['returns'].rolling(50).std() * np.sqrt(252)
        
        # Volatility of volatility
        features['vol_of_vol_20d'] = features['realized_vol_20d'].rolling(20).std()
        
        # High-low range
        if 'high' in data.columns and 'low' in data.columns:
            features['hl_range'] = (data['high'] - data['low']) / data['close']
            features['hl_range_20d_avg'] = features['hl_range'].rolling(20).mean()
    
    if feature_set == 'comprehensive':
        # ====================================================================
        # 3. TREND FEATURES
        # ====================================================================
        # Moving averages
        sma_20 = data['close'].rolling(20).mean()
        sma_50 = data['close'].rolling(50).mean()
        sma_200 = data['close'].rolling(200).mean()
        
        # Price distance from moving averages
        features['price_sma20_ratio'] = data['close'] / sma_20
        features['price_sma50_ratio'] = data['close'] / sma_50
        features['price_sma200_ratio'] = data['close'] / sma_200
        
        # SMA relationships
        features['sma20_sma50_ratio'] = sma_20 / sma_50
        features['sma50_sma200_ratio'] = sma_50 / sma_200
        
        # ADX (Average Directional Index) - simplified approximation
        if 'high' in data.columns and 'low' in data.columns:
            plus_dm = data['high'].diff()
            minus_dm = -data['low'].diff()
            plus_dm = plus_dm.where(plus_dm > minus_dm, 0)
            minus_dm = minus_dm.where(minus_dm > plus_dm, 0)
            tr = np.maximum(data['high'] - data['low'],
                           np.maximum(np.abs(data['high'] - data['close'].shift(1)),
                                     np.abs(data['low'] - data['close'].shift(1))))
            atr = tr.rolling(14).mean()
            plus_di = 100 * (plus_dm.rolling(14).mean() / atr)
            minus_di = 100 * (minus_dm.rolling(14).mean() / atr)
            dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di)
            features['adx_14'] = dx.rolling(14).mean()
        
        # Linear regression slope (trend strength)
        def calc_slope(series, window=20):
            def slope_func(x):
                if len(x) < window or x.isna().any():
                    return np.nan
                y = x.values
                x_vals = np.arange(len(y))
                try:
                    slope, _ = np.polyfit(x_vals, y, 1)
                    return slope
                except:
                    return np.nan
            return series.rolling(window).apply(slope_func, raw=False)
        
        features['trend_slope_20d'] = calc_slope(data['close'], 20)
        features['trend_slope_50d'] = calc_slope(data['close'], 50)
        
        # ====================================================================
        # 4. MOMENTUM FEATURES
        # ====================================================================
        # RSI (Relative Strength Index)
        delta = data['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        features['rsi_14'] = 100 - (100 / (1 + rs))
        
        # MACD
        ema_12 = data['close'].ewm(span=12, adjust=False).mean()
        ema_26 = data['close'].ewm(span=26, adjust=False).mean()
        features['macd'] = ema_12 - ema_26
        features['macd_signal'] = features['macd'].ewm(span=9, adjust=False).mean()
        features['macd_histogram'] = features['macd'] - features['macd_signal']
        
        # Rate of change
        features['roc_5d'] = data['close'].pct_change(5)
        features['roc_10d'] = data['close'].pct_change(10)
        features['roc_20d'] = data['close'].pct_change(20)
        
        # ====================================================================
        # 5. VOLUME FEATURES (if available)
        # ====================================================================
        if 'volume' in data.columns:
            volume_sma_20 = data['volume'].rolling(20).mean()
            features['volume_ratio'] = data['volume'] / volume_sma_20
            features['volume_trend'] = data['volume'].rolling(5).mean() / volume_sma_20
    
    # Drop intermediate SMA columns if they exist
    cols_to_drop = [c for c in features.columns if c.startswith('sma_')]
    features = features.drop(columns=cols_to_drop, errors='ignore')
    
    return features.dropna()


def normalize_features(features: pd.DataFrame, method: str = 'standard') -> Tuple[pd.DataFrame, object]:
    """
    Normalize features using StandardScaler or RobustScaler.
    
    Parameters:
    -----------
    features : pd.DataFrame
        Feature DataFrame
    method : str
        'standard' or 'robust'
        
    Returns:
    --------
    Tuple[pd.DataFrame, object]
        Normalized features and fitted scaler
    """
    if method == 'robust':
        scaler = RobustScaler()
    else:
        scaler = StandardScaler()
    
    scaled_data = scaler.fit_transform(features)
    scaled_df = pd.DataFrame(scaled_data, index=features.index, columns=features.columns)
    
    return scaled_df, scaler


# Engineer features for market, sector, and asset
print("=" * 60)
print("FEATURE ENGINEERING FOR REGIME DETECTION")
print("=" * 60)

market_features = engineer_regime_features(df_market, 'comprehensive')
print(f"\nMarket features: {market_features.shape[0]} observations, {market_features.shape[1]} features")

sector_features = engineer_regime_features(df_sector, 'comprehensive')
print(f"Sector features: {sector_features.shape[0]} observations, {sector_features.shape[1]} features")

asset_features = engineer_regime_features(df_asset, 'comprehensive')
print(f"Asset features: {asset_features.shape[0]} observations, {asset_features.shape[1]} features")

print(f"\nFeature columns ({len(market_features.columns)}):")
for i, col in enumerate(market_features.columns):
    category = 'Return' if 'return' in col.lower() else \
               'Volatility' if 'vol' in col.lower() or 'hl_range' in col.lower() else \
               'Trend' if 'sma' in col.lower() or 'slope' in col.lower() or 'adx' in col.lower() else \
               'Momentum' if 'rsi' in col.lower() or 'macd' in col.lower() or 'roc' in col.lower() else \
               'Volume' if 'volume' in col.lower() else 'Other'
    print(f"  [{category}] {col}")

### 6B. Individual Clustering Methods

**Method 1 - Gaussian Mixture Models (GMM):**
- Use sklearn's GaussianMixture
- Test 3-6 components (regimes)
- Use BIC (Bayesian Information Criterion) for optimal component selection
- Captures probabilistic cluster assignments and elliptical cluster shapes
- Extract soft cluster probabilities for each time point
- Advantages: Provides probability distributions, handles overlapping clusters

**Method 2 - K-Means Clustering:**
- Use sklearn's KMeans
- Test k=3 to k=6 clusters
- Use silhouette score and elbow method for optimal k selection
- Provides hard cluster assignments
- Advantages: Fast, simple, works well for spherical clusters

**Method 3 - Agglomerative (Hierarchical) Clustering:**
- Use sklearn's AgglomerativeClustering
- Test 3-6 clusters with different linkage methods (ward, complete, average)
- Create dendrogram to visualize hierarchical relationships
- Advantages: No assumption about cluster shape, reveals hierarchical structure

**Method 4 - Change-Point Detection (CPD):**
- Use ruptures library (Pelt, Binseg, or BottomUp algorithms)
- Detect structural breaks in the feature time series
- Identifies when statistical properties of data change
- Use detected change points to segment time series into regimes
- Advantages: Explicitly detects regime transitions, doesn't assume fixed number of regimes

In [ ]:
# ============================================================================
# SECTION 6B: INDIVIDUAL CLUSTERING METHODS
# ============================================================================

def fit_gmm_regimes(features: pd.DataFrame, n_components_range: range = range(3, 7),
                    random_state: int = 42) -> Tuple[np.ndarray, GaussianMixture, int, np.ndarray]:
    """
    Fit Gaussian Mixture Model for regime detection.
    Uses BIC (Bayesian Information Criterion) for optimal component selection.
    
    Parameters:
    -----------
    features : pd.DataFrame
        Normalized feature DataFrame
    n_components_range : range
        Range of components to test (default: 3-6)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    Tuple[np.ndarray, GaussianMixture, int, np.ndarray]
        Labels, fitted model, optimal n_components, soft probabilities
    """
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    # Find optimal n_components using BIC
    bic_scores = []
    aic_scores = []
    models = []
    
    print("GMM: Testing components...")
    for n in n_components_range:
        gmm = GaussianMixture(n_components=n, random_state=random_state, n_init=5,
                              covariance_type='full', max_iter=200)
        gmm.fit(X_scaled)
        bic_scores.append(gmm.bic(X_scaled))
        aic_scores.append(gmm.aic(X_scaled))
        models.append(gmm)
        print(f"  n={n}: BIC={bic_scores[-1]:.0f}, AIC={aic_scores[-1]:.0f}")
    
    # Select model with lowest BIC
    best_idx = np.argmin(bic_scores)
    best_n = list(n_components_range)[best_idx]
    best_model = models[best_idx]
    
    # Get labels and soft probabilities
    labels = best_model.predict(X_scaled)
    probabilities = best_model.predict_proba(X_scaled)
    
    print(f"GMM: Optimal components = {best_n} (BIC = {bic_scores[best_idx]:.0f})")
    
    return labels, best_model, best_n, probabilities


def fit_kmeans_regimes(features: pd.DataFrame, k_range: range = range(3, 7),
                       random_state: int = 42) -> Tuple[np.ndarray, KMeans, int, np.ndarray]:
    """
    Fit K-Means clustering for regime detection.
    Uses silhouette score and elbow method for optimal k selection.
    
    Parameters:
    -----------
    features : pd.DataFrame
        Feature DataFrame
    k_range : range
        Range of k values to test
    random_state : int
        Random seed
        
    Returns:
    --------
    Tuple[np.ndarray, KMeans, int, np.ndarray]
        Labels, fitted model, optimal k, distances to cluster centers
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    silhouette_scores = []
    inertias = []  # For elbow method
    models = []
    
    print("K-Means: Testing cluster counts...")
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10, max_iter=300)
        labels = kmeans.fit_predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        silhouette_scores.append(score)
        inertias.append(kmeans.inertia_)
        models.append(kmeans)
        print(f"  k={k}: Silhouette={score:.3f}, Inertia={kmeans.inertia_:.0f}")
    
    # Select best k by silhouette score
    best_idx = np.argmax(silhouette_scores)
    best_k = list(k_range)[best_idx]
    best_model = models[best_idx]
    labels = best_model.predict(X_scaled)
    
    # Calculate distance to cluster centers (inverse for confidence)
    distances = best_model.transform(X_scaled)
    min_distances = distances.min(axis=1)
    
    print(f"K-Means: Optimal k = {best_k} (Silhouette = {silhouette_scores[best_idx]:.3f})")
    
    return labels, best_model, best_k, min_distances


def fit_agglomerative_regimes(features: pd.DataFrame, n_clusters_range: range = range(3, 7),
                               linkage_methods: list = None) -> Tuple[np.ndarray, int, str]:
    """
    Fit Agglomerative (Hierarchical) clustering.
    Tests multiple linkage methods and cluster counts.
    
    Parameters:
    -----------
    features : pd.DataFrame
        Feature DataFrame
    n_clusters_range : range
        Range of cluster counts to test
    linkage_methods : list
        Linkage methods to test (default: ['ward', 'complete', 'average'])
        
    Returns:
    --------
    Tuple[np.ndarray, int, str]
        Best labels, optimal n_clusters, best linkage method
    """
    if linkage_methods is None:
        linkage_methods = ['ward', 'complete', 'average']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    best_score = -1
    best_labels = None
    best_n = None
    best_linkage = None
    
    print("Agglomerative: Testing configurations...")
    for linkage in linkage_methods:
        for n in n_clusters_range:
            try:
                agg = AgglomerativeClustering(n_clusters=n, linkage=linkage)
                labels = agg.fit_predict(X_scaled)
                score = silhouette_score(X_scaled, labels)
                
                if score > best_score:
                    best_score = score
                    best_labels = labels
                    best_n = n
                    best_linkage = linkage
                
                print(f"  {linkage}, n={n}: Silhouette={score:.3f}")
            except Exception as e:
                continue
    
    print(f"Agglomerative: Optimal = {best_n} clusters, {best_linkage} linkage (Silhouette = {best_score:.3f})")
    
    return best_labels, best_n, best_linkage


def detect_change_points(features: pd.DataFrame, method: str = 'pelt',
                         n_bkps: int = None, penalty: int = 3) -> Tuple[np.ndarray, list]:
    """
    Detect structural breaks using change-point detection (ruptures library).
    
    Parameters:
    -----------
    features : pd.DataFrame
        Feature DataFrame
    method : str
        CPD algorithm: 'pelt', 'binseg', or 'bottomup'
    n_bkps : int
        Number of breakpoints (for binseg/bottomup). If None, uses penalty.
    penalty : int
        Penalty parameter for PELT
        
    Returns:
    --------
    Tuple[np.ndarray, list]
        Regime labels based on segments, list of change point indices
    """
    if not RUPTURES_AVAILABLE:
        print("Warning: ruptures library not available, returning uniform segments")
        return np.zeros(len(features)), []
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    print(f"Change-Point Detection: Using {method.upper()} algorithm...")
    
    try:
        if method == 'pelt':
            algo = rpt.Pelt(model='rbf', min_size=10).fit(X_scaled)
            result = algo.predict(pen=penalty)
        elif method == 'binseg':
            n_breakpoints = n_bkps if n_bkps else min(10, len(features) // 50)
            algo = rpt.Binseg(model='rbf', min_size=10).fit(X_scaled)
            result = algo.predict(n_bkps=n_breakpoints)
        else:  # bottomup
            n_breakpoints = n_bkps if n_bkps else min(10, len(features) // 50)
            algo = rpt.BottomUp(model='rbf', min_size=10).fit(X_scaled)
            result = algo.predict(n_bkps=n_breakpoints)
    except Exception as e:
        print(f"  CPD failed: {e}")
        return np.zeros(len(features)), []
    
    # Convert change points to labels
    labels = np.zeros(len(features), dtype=int)
    prev_bkp = 0
    for i, bkp in enumerate(result):
        bkp = min(bkp, len(features))  # Ensure within bounds
        labels[prev_bkp:bkp] = i
        prev_bkp = bkp
    
    print(f"  Detected {len(result)} change points: {result[:10]}...")
    
    return labels, result


# ============================================================================
# CLUSTERING VALIDATION METRICS
# ============================================================================

def validate_clustering(features: pd.DataFrame, labels: np.ndarray, method_name: str) -> dict:
    """
    Calculate validation metrics for clustering results.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    metrics = {
        'method': method_name,
        'n_clusters': len(np.unique(labels)),
        'silhouette': silhouette_score(X_scaled, labels),
        'davies_bouldin': davies_bouldin_score(X_scaled, labels),
        'calinski_harabasz': calinski_harabasz_score(X_scaled, labels)
    }
    
    return metrics


print("Individual clustering methods defined:")
print("  - fit_gmm_regimes(): GMM with BIC selection")
print("  - fit_kmeans_regimes(): K-Means with silhouette selection")
print("  - fit_agglomerative_regimes(): Hierarchical with multiple linkages")
print("  - detect_change_points(): CPD using ruptures library")
print("  - validate_clustering(): Validation metrics calculation")

### 6C. Ensemble Regime Classification

**Voting/Consensus Mechanism:**
For each time point, collect regime labels from all 4 methods and implement weighted voting:

- **GMM:** Weight by probability confidence (entropy-based)
- **K-Means:** Weight by distance to cluster center (inverse)
- **Agglomerative:** Weight by dendrogram height/distance
- **CPD:** Higher weight near detected change points

**Ensemble Strategies:**
1. **Majority Voting:** Most common regime label wins
2. **Weighted Voting:** Weight each method's vote by its confidence metric
3. **Hierarchical Ensemble:** Use CPD to identify major regime boundaries, then use GMM/K-Means/Agglomerative within each segment
4. **Stacking:** Use cluster labels as features for meta-classifier

**Regime Label Mapping:**
After ensemble classification, map numeric cluster labels to meaningful regime names using feature centroids:
- High returns + low volatility → "Bull_Low_Vol"
- High returns + high volatility → "Bull_High_Vol"
- Negative returns + high volatility → "Bear"
- Low returns + low volatility → "Choppy" or "Sideways"

In [ ]:
# ============================================================================
# SECTION 6C: ENSEMBLE REGIME CLASSIFICATION
# ============================================================================

def ensemble_regime_classification(gmm_labels: np.ndarray, kmeans_labels: np.ndarray,
                                   agg_labels: np.ndarray, cpd_labels: np.ndarray,
                                   gmm_probs: np.ndarray = None,
                                   kmeans_distances: np.ndarray = None,
                                   weights: dict = None,
                                   strategy: str = 'weighted') -> Tuple[np.ndarray, np.ndarray]:
    """
    Combine multiple clustering methods using voting/consensus.
    
    Parameters:
    -----------
    gmm_labels : np.ndarray
        Labels from GMM
    kmeans_labels : np.ndarray
        Labels from K-Means
    agg_labels : np.ndarray
        Labels from Agglomerative clustering
    cpd_labels : np.ndarray
        Labels from Change-Point Detection
    gmm_probs : np.ndarray
        Soft probabilities from GMM
    kmeans_distances : np.ndarray
        Distances to cluster centers from K-Means
    weights : dict
        Weights for each method (default: equal weights)
    strategy : str
        'majority', 'weighted', 'hierarchical', or 'stacking'
        
    Returns:
    --------
    Tuple[np.ndarray, np.ndarray]
        Ensemble labels, confidence scores
    """
    n_samples = len(gmm_labels)
    
    # Default weights
    if weights is None:
        weights = {'gmm': 0.30, 'kmeans': 0.25, 'agg': 0.25, 'cpd': 0.20}
    
    # Calculate confidence weights for each method
    gmm_conf = np.ones(n_samples)
    kmeans_conf = np.ones(n_samples)
    
    if gmm_probs is not None:
        # GMM confidence: 1 - entropy of probabilities
        entropy = -np.sum(gmm_probs * np.log(gmm_probs + 1e-10), axis=1)
        max_entropy = np.log(gmm_probs.shape[1])
        gmm_conf = 1 - (entropy / max_entropy)
    
    if kmeans_distances is not None:
        # K-Means confidence: inverse of min distance (normalized)
        kmeans_conf = 1 / (1 + kmeans_distances)
        kmeans_conf = kmeans_conf / kmeans_conf.max()
    
    if strategy == 'majority':
        # Simple majority voting
        all_labels = np.column_stack([gmm_labels, kmeans_labels, agg_labels, cpd_labels])
        ensemble_labels = np.zeros(n_samples, dtype=int)
        
        for i in range(n_samples):
            votes = all_labels[i, :]
            unique, counts = np.unique(votes, return_counts=True)
            ensemble_labels[i] = unique[np.argmax(counts)]
        
        # Confidence = fraction of methods that agree
        agreement = np.zeros(n_samples)
        for i in range(n_samples):
            votes = all_labels[i, :]
            mode = ensemble_labels[i]
            agreement[i] = np.sum(votes == mode) / len(votes)
        confidence = agreement
        
    elif strategy == 'weighted':
        # Weighted voting based on method confidence
        n_unique = max(len(np.unique(l)) for l in [gmm_labels, kmeans_labels, agg_labels, cpd_labels])
        
        # Create weighted vote matrix
        vote_matrix = np.zeros((n_samples, n_unique))
        
        for i in range(n_samples):
            # Add weighted votes
            if gmm_labels[i] < n_unique:
                vote_matrix[i, gmm_labels[i]] += weights['gmm'] * gmm_conf[i]
            if kmeans_labels[i] < n_unique:
                vote_matrix[i, kmeans_labels[i]] += weights['kmeans'] * kmeans_conf[i]
            if agg_labels[i] < n_unique:
                vote_matrix[i, agg_labels[i]] += weights['agg']
            if cpd_labels[i] < n_unique:
                vote_matrix[i, cpd_labels[i]] += weights['cpd']
        
        ensemble_labels = np.argmax(vote_matrix, axis=1)
        
        # Confidence = normalized max vote weight
        max_votes = vote_matrix.max(axis=1)
        total_votes = vote_matrix.sum(axis=1)
        confidence = max_votes / (total_votes + 1e-10)
        
    elif strategy == 'hierarchical':
        # Use CPD for major boundaries, then use GMM within segments
        ensemble_labels = gmm_labels.copy()
        
        # At CPD change points, trust CPD more
        cpd_changes = np.where(np.diff(cpd_labels) != 0)[0]
        for cp in cpd_changes:
            # Smooth transition at change points
            window = slice(max(0, cp-2), min(n_samples, cp+3))
            ensemble_labels[window] = cpd_labels[window]
        
        confidence = gmm_conf
        
    else:  # stacking
        # Use cluster labels as meta-features
        meta_features = np.column_stack([gmm_labels, kmeans_labels, agg_labels, cpd_labels])
        # Simple approach: weighted average
        ensemble_labels = np.round(
            weights['gmm'] * gmm_labels + 
            weights['kmeans'] * kmeans_labels + 
            weights['agg'] * agg_labels + 
            weights['cpd'] * cpd_labels
        ).astype(int)
        confidence = gmm_conf
    
    return ensemble_labels, confidence


def map_regime_labels(labels: np.ndarray, features: pd.DataFrame,
                      key_features: list = None) -> np.ndarray:
    """
    Map numeric cluster labels to meaningful regime names.
    Uses feature centroids to interpret regimes.
    
    Parameters:
    -----------
    labels : np.ndarray
        Cluster labels
    features : pd.DataFrame
        Feature DataFrame
    key_features : list
        Key features for regime interpretation (default: returns and volatility)
        
    Returns:
    --------
    np.ndarray
        Array of regime name strings
    """
    if key_features is None:
        # Find returns and volatility columns
        return_col = [c for c in features.columns if 'returns_20d' in c.lower()]
        vol_col = [c for c in features.columns if 'realized_vol_20d' in c.lower() or 'vol_20d' in c.lower()]
        key_features = (return_col[:1] if return_col else []) + (vol_col[:1] if vol_col else [])
    
    if not key_features:
        # Default to first two features
        key_features = features.columns[:2].tolist()
    
    # Calculate centroids
    regime_names = []
    
    for label in np.unique(labels):
        mask = labels == label
        centroid = features.loc[features.index[mask], key_features].mean()
        
        # Interpret centroid
        ret_val = centroid.iloc[0] if len(centroid) > 0 else 0
        vol_val = centroid.iloc[1] if len(centroid) > 1 else 0
        
        if ret_val > 0.01:  # Positive returns
            if vol_val > 0.20:
                name = 'Bull_High_Vol'
            else:
                name = 'Bull_Low_Vol'
        elif ret_val < -0.01:  # Negative returns
            name = 'Bear'
        else:  # Low returns
            if vol_val > 0.20:
                name = 'Choppy'
            else:
                name = 'Sideways'
        
        regime_names.append((label, name))
    
    # Create mapping and apply
    label_to_name = {label: name for label, name in regime_names}
    return np.array([label_to_name.get(l, f'Regime_{l}') for l in labels])


def calculate_regime_transitions(regime_series: pd.Series) -> pd.DataFrame:
    """
    Calculate transition probability matrix for regimes.
    
    Returns:
    --------
    pd.DataFrame
        Transition probability matrix
    """
    regimes = regime_series.unique()
    n_regimes = len(regimes)
    
    # Count transitions
    transition_counts = pd.DataFrame(0, index=regimes, columns=regimes)
    
    for i in range(len(regime_series) - 1):
        from_regime = regime_series.iloc[i]
        to_regime = regime_series.iloc[i + 1]
        transition_counts.loc[from_regime, to_regime] += 1
    
    # Convert to probabilities
    transition_probs = transition_counts.div(transition_counts.sum(axis=1), axis=0)
    transition_probs = transition_probs.fillna(0)
    
    return transition_probs


print("Ensemble classification functions defined:")
print("  - ensemble_regime_classification(): Combine methods with voting")
print("  - map_regime_labels(): Map to meaningful regime names")
print("  - calculate_regime_transitions(): Transition probability matrix")

### 6D. Regime Detection Validation & Visualization

**Validation Metrics:**
- Silhouette score: Measure cluster cohesion and separation
- Davies-Bouldin index: Lower is better
- Calinski-Harabasz index: Higher is better
- Stability analysis: Bootstrap resampling to test regime stability
- Transition frequency: Regimes shouldn't change too frequently (apply smoothing if needed)
- Economic interpretation: Do regimes align with known market events?

In [ ]:
# ============================================================================
# SECTION 6D: REGIME DETECTION VALIDATION & VISUALIZATION
# ============================================================================

def validate_regime_clustering_comprehensive(features: pd.DataFrame, labels: np.ndarray,
                                             n_bootstrap: int = 100) -> dict:
    """
    Comprehensive validation including stability analysis via bootstrap.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    # Standard clustering metrics
    metrics = {
        'silhouette': silhouette_score(X_scaled, labels),
        'davies_bouldin': davies_bouldin_score(X_scaled, labels),
        'calinski_harabasz': calinski_harabasz_score(X_scaled, labels),
        'n_clusters': len(np.unique(labels)),
        'cluster_sizes': dict(zip(*np.unique(labels, return_counts=True)))
    }
    
    # Bootstrap stability analysis (vectorized)
    n_samples = len(labels)
    bootstrap_silhouettes = []
    rng = np.random.default_rng(RANDOM_SEED)
    
    for _ in range(n_bootstrap):
        # Sample with replacement
        idx = rng.choice(n_samples, size=n_samples, replace=True)
        X_boot = X_scaled[idx]
        labels_boot = labels[idx]
        
        # Only calculate if we have multiple clusters
        if len(np.unique(labels_boot)) > 1:
            bootstrap_silhouettes.append(silhouette_score(X_boot, labels_boot))
    
    metrics['silhouette_std'] = np.std(bootstrap_silhouettes) if bootstrap_silhouettes else 0
    metrics['silhouette_95ci'] = (np.percentile(bootstrap_silhouettes, 2.5),
                                   np.percentile(bootstrap_silhouettes, 97.5)) if bootstrap_silhouettes else (0, 0)
    
    # Transition frequency analysis
    transitions = np.sum(np.diff(labels) != 0)
    metrics['transition_count'] = transitions
    metrics['avg_regime_duration'] = n_samples / (transitions + 1)  # Average days per regime
    
    return metrics


def apply_regime_smoothing(labels: np.ndarray, min_duration: int = 3) -> np.ndarray:
    """
    Apply smoothing to prevent too frequent regime changes.
    Vectorized implementation.
    """
    smoothed = labels.copy()
    n = len(smoothed)
    
    # Find regime change points
    change_points = np.where(np.diff(smoothed) != 0)[0] + 1
    change_points = np.concatenate([[0], change_points, [n]])
    
    # Merge short segments
    for i in range(1, len(change_points) - 1):
        segment_len = change_points[i] - change_points[i-1]
        if segment_len < min_duration:
            # Merge with previous segment
            smoothed[change_points[i-1]:change_points[i]] = smoothed[change_points[i-1] - 1] if change_points[i-1] > 0 else smoothed[change_points[i]]
    
    return smoothed


def check_economic_interpretation(regime_series: pd.Series, price_data: pd.DataFrame,
                                  known_events: dict = None) -> pd.DataFrame:
    """
    Check if regimes align with known market events.
    known_events format: {'2020-03': 'COVID Crash', '2022-01': 'Fed Tightening'}
    """
    if known_events is None:
        known_events = {
            '2020-02': 'COVID Crash Start',
            '2020-03': 'COVID Bottom',
            '2020-04': 'Recovery Begin',
            '2021-11': 'Market Peak',
            '2022-01': 'Fed Tightening',
            '2022-06': 'Bear Market',
            '2022-10': 'Market Bottom',
            '2023-03': 'Banking Crisis',
        }
    
    results = []
    for date_str, event in known_events.items():
        try:
            # Find nearest date in regime series
            target_date = pd.to_datetime(date_str)
            mask = regime_series.index >= target_date
            if mask.any():
                nearest_idx = regime_series.index[mask][0]
                regime_at_event = regime_series.loc[nearest_idx]
                results.append({'Event': event, 'Date': date_str, 'Regime': regime_at_event})
        except:
            continue
    
    return pd.DataFrame(results)


# Run comprehensive validation
print("="*70)
print("COMPREHENSIVE REGIME VALIDATION")
print("="*70)

market_validation = validate_regime_clustering_comprehensive(market_features, ensemble_labels_mkt)
print(f"\nMARKET REGIMES ({MARKET_PROXY}):")
print(f"  Silhouette Score: {market_validation['silhouette']:.3f} ± {market_validation['silhouette_std']:.3f}")
print(f"  95% CI: [{market_validation['silhouette_95ci'][0]:.3f}, {market_validation['silhouette_95ci'][1]:.3f}]")
print(f"  Davies-Bouldin Index: {market_validation['davies_bouldin']:.3f} (lower is better)")
print(f"  Calinski-Harabasz Index: {market_validation['calinski_harabasz']:.1f} (higher is better)")
print(f"  Transition Count: {market_validation['transition_count']}")
print(f"  Avg Regime Duration: {market_validation['avg_regime_duration']:.1f} days")

sector_validation = validate_regime_clustering_comprehensive(sector_features, ensemble_labels_sec)
print(f"\nSECTOR REGIMES ({SECTOR_ETF}):")
print(f"  Silhouette Score: {sector_validation['silhouette']:.3f} ± {sector_validation['silhouette_std']:.3f}")
print(f"  Davies-Bouldin Index: {sector_validation['davies_bouldin']:.3f}")
print(f"  Avg Regime Duration: {sector_validation['avg_regime_duration']:.1f} days")

asset_validation = validate_regime_clustering_comprehensive(asset_features, ensemble_labels_ast)
print(f"\nASSET REGIMES ({TICKER}):")
print(f"  Silhouette Score: {asset_validation['silhouette']:.3f} ± {asset_validation['silhouette_std']:.3f}")
print(f"  Davies-Bouldin Index: {asset_validation['davies_bouldin']:.3f}")
print(f"  Avg Regime Duration: {asset_validation['avg_regime_duration']:.1f} days")

# Economic interpretation check
print("\nECONOMIC EVENT ALIGNMENT:")
econ_check = check_economic_interpretation(market_regimes, df_market)
if len(econ_check) > 0:
    print(econ_check.to_string(index=False))

#### 6D.1 Ensemble Method Visualizations

In [ ]:
# ============================================================================
# COMPARISON PLOT: Regime assignments from each method
# ============================================================================

def plot_regime_comparison(gmm_labels, kmeans_labels, agg_labels, cpd_labels,
                           ensemble_labels, index, title, figsize=(14, 12)):
    """
    Show regime assignments from each individual method as separate time series rows,
    with ensemble result at bottom.
    """
    fig, axes = plt.subplots(5, 1, figsize=figsize, sharex=True)
    
    methods = [
        ('GMM', gmm_labels, '#3498db'),
        ('K-Means', kmeans_labels, '#e74c3c'),
        ('Agglomerative', agg_labels, '#2ecc71'),
        ('CPD', cpd_labels, '#9b59b6'),
        ('ENSEMBLE', ensemble_labels, '#2c3e50')  # Emphasized
    ]
    
    for ax, (method, labels, color) in zip(axes, methods):
        ax.plot(index, labels, drawstyle='steps-post', color=color, linewidth=1.5)
        ax.fill_between(index, labels, alpha=0.3, step='post', color=color)
        ax.set_ylabel(method, fontsize=11, fontweight='bold' if method == 'ENSEMBLE' else 'normal')
        ax.grid(True, alpha=0.3)
        ax.set_yticks(range(int(max(labels)) + 1))
        
        # Highlight ensemble row
        if method == 'ENSEMBLE':
            ax.set_facecolor('#f5f5f5')
            ax.spines['left'].set_linewidth(3)
            ax.spines['left'].set_color(color)
    
    axes[0].set_title(title, fontsize=14, fontweight='bold')
    axes[-1].set_xlabel('Date', fontsize=12)
    
    plt.tight_layout()
    plt.show()


# Plot method comparison for market
plot_regime_comparison(
    gmm_labels_mkt, kmeans_labels_mkt, agg_labels_mkt, cpd_labels_mkt,
    ensemble_labels_mkt, market_features.index,
    f'{MARKET_PROXY} - Regime Method Comparison'
)

In [ ]:
# ============================================================================
# CONSENSUS HEATMAP: Agreement level between methods over time
# ============================================================================

def plot_ensemble_consensus_heatmap(gmm_labels, kmeans_labels, agg_labels, cpd_labels,
                                    index, title, figsize=(14, 5)):
    """
    Show agreement level between methods over time (darker = higher agreement).
    """
    n_samples = len(gmm_labels)
    
    # Calculate agreement scores vectorized
    labels_matrix = np.vstack([gmm_labels, kmeans_labels, agg_labels, cpd_labels]).T
    agreement = np.zeros(n_samples)
    
    for i in range(n_samples):
        unique, counts = np.unique(labels_matrix[i], return_counts=True)
        agreement[i] = counts.max() / 4  # Max agreement out of 4 methods
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create colormap based on agreement
    colors = plt.cm.RdYlGn(agreement)  # Red=low, Yellow=medium, Green=high
    
    # Create bar chart with colors
    for i, (date, agr) in enumerate(zip(index, agreement)):
        ax.bar(date, 1, width=1, color=colors[i], alpha=0.8)
    
    ax.axhline(y=0.75, color='darkgreen', linestyle='--', linewidth=2, label='High agreement (75%+)')
    ax.axhline(y=0.50, color='orange', linestyle='--', linewidth=2, label='Moderate (50%)')
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Agreement Level', fontsize=12)
    ax.set_ylim(0, 1.1)
    
    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.RdYlGn, norm=plt.Normalize(0.25, 1))
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label('Agreement', fontsize=11)
    
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"Agreement Statistics:")
    print(f"  Mean agreement: {np.mean(agreement):.1%}")
    print(f"  High agreement periods (>=75%): {(agreement >= 0.75).sum() / len(agreement):.1%}")
    print(f"  Low agreement periods (<50%): {(agreement < 0.5).sum() / len(agreement):.1%}")


plot_ensemble_consensus_heatmap(
    gmm_labels_mkt, kmeans_labels_mkt, agg_labels_mkt, cpd_labels_mkt,
    market_features.index,
    f'{MARKET_PROXY} - Method Consensus Over Time'
)

In [ ]:
# ============================================================================
# FEATURE SPACE VISUALIZATION: 2D PCA projection colored by regime
# ============================================================================

def plot_feature_space_pca(features: pd.DataFrame, regime_labels: np.ndarray,
                           regime_names: pd.Series, title: str, figsize=(12, 10)):
    """
    2D PCA projection of features, colored by ensemble regime assignment.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    
    fig, ax = plt.subplots(figsize=figsize)
    
    # Plot each regime with its color
    unique_regimes = regime_names.unique()
    
    for regime in unique_regimes:
        mask = regime_names.values == regime
        color = get_regime_color(regime)
        ax.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                  c=color, label=regime, alpha=0.6, s=30, edgecolors='white', linewidth=0.5)
    
    # Add cluster centers
    for regime in unique_regimes:
        mask = regime_names.values == regime
        center = X_pca[mask].mean(axis=0)
        color = get_regime_color(regime)
        ax.scatter(center[0], center[1], c=color, s=300, marker='*', 
                  edgecolors='black', linewidth=2, zorder=5)
    
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(title='Regime', loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPCA Explained Variance: PC1={pca.explained_variance_ratio_[0]:.1%}, PC2={pca.explained_variance_ratio_[1]:.1%}")
    print(f"Total variance explained: {sum(pca.explained_variance_ratio_):.1%}")


plot_feature_space_pca(
    market_features, ensemble_labels_mkt, market_regimes,
    f'{MARKET_PROXY} - Feature Space (PCA) by Regime'
)

In [ ]:
# ============================================================================
# METHOD AGREEMENT MATRIX: How often do pairs of methods agree?
# ============================================================================

def plot_method_agreement_matrix(gmm_labels, kmeans_labels, agg_labels, cpd_labels,
                                  title: str, figsize=(8, 7)):
    """
    Show how often pairs of methods agree on regime classification.
    """
    methods = ['GMM', 'K-Means', 'Agglomerative', 'CPD']
    labels_list = [gmm_labels, kmeans_labels, agg_labels, cpd_labels]
    n_methods = len(methods)
    
    # Calculate pairwise agreement (vectorized)
    agreement_matrix = np.zeros((n_methods, n_methods))
    
    for i in range(n_methods):
        for j in range(n_methods):
            agreement_matrix[i, j] = np.mean(labels_list[i] == labels_list[j])
    
    fig, ax = plt.subplots(figsize=figsize)
    
    im = ax.imshow(agreement_matrix, cmap='RdYlGn', vmin=0, vmax=1)
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Agreement Rate', fontsize=11)
    
    # Add labels
    ax.set_xticks(range(n_methods))
    ax.set_yticks(range(n_methods))
    ax.set_xticklabels(methods, fontsize=11)
    ax.set_yticklabels(methods, fontsize=11)
    
    # Add annotations
    for i in range(n_methods):
        for j in range(n_methods):
            text = f'{agreement_matrix[i, j]:.0%}'
            color = 'white' if agreement_matrix[i, j] < 0.5 else 'black'
            ax.text(j, i, text, ha='center', va='center', fontsize=12, 
                   fontweight='bold', color=color)
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate overall method concordance
    off_diag = agreement_matrix[~np.eye(n_methods, dtype=bool)]
    print(f"\nOverall Method Agreement: {off_diag.mean():.1%}")
    print(f"Min pairwise agreement: {off_diag.min():.1%}")
    print(f"Max pairwise agreement: {off_diag.max():.1%}")


plot_method_agreement_matrix(
    gmm_labels_mkt, kmeans_labels_mkt, agg_labels_mkt, cpd_labels_mkt,
    f'{MARKET_PROXY} - Method Agreement Matrix'
)

### 6E. Apply Ensemble Regime Detection to Three Levels

Apply the ensemble method to Market, Sector, and Asset levels with priority visualizations.

In [ ]:
# ============================================================================
# SECTOR ETF MAPPING
# ============================================================================

SECTOR_ETF_MAP = {
    # Technology
    'XLK': 'Technology',
    'QQQ': 'Technology',
    'VGT': 'Technology',
    # Healthcare
    'XLV': 'Healthcare',
    'VHT': 'Healthcare',
    'IBB': 'Biotechnology',
    # Financials
    'XLF': 'Financials',
    'VFH': 'Financials',
    # Consumer
    'XLY': 'Consumer Discretionary',
    'XLP': 'Consumer Staples',
    # Energy
    'XLE': 'Energy',
    'VDE': 'Energy',
    # Industrials
    'XLI': 'Industrials',
    'VIS': 'Industrials',
    # Real Estate
    'XLRE': 'Real Estate',
    'VNQ': 'Real Estate',
    # Materials
    'XLB': 'Materials',
    # Utilities
    'XLU': 'Utilities',
    # Communication
    'XLC': 'Communication Services',
}

# Reverse mapping: sector name to primary ETF
SECTOR_TO_ETF = {
    'Technology': 'XLK',
    'Healthcare': 'XLV',
    'Biotechnology': 'XLV',  # Fall back to healthcare
    'Financials': 'XLF',
    'Consumer Discretionary': 'XLY',
    'Consumer Staples': 'XLP',
    'Energy': 'XLE',
    'Industrials': 'XLI',
    'Real Estate': 'XLRE',
    'Materials': 'XLB',
    'Utilities': 'XLU',
    'Communication Services': 'XLC',
}

def get_sector_etf(ticker: str, qb) -> str:
    """
    Dynamically determine sector ETF based on ticker's sector classification.
    Uses QuantConnect's fundamental data.
    """
    try:
        # Get security fundamentals
        symbol = qb.AddEquity(ticker).Symbol
        fundamentals = qb.GetFundamental(symbol, 'AssetClassification.MorningstarSectorCode')
        
        sector_code_map = {
            311: 'XLK',   # Technology
            206: 'XLV',   # Healthcare
            103: 'XLF',   # Financial Services
            102: 'XLY',   # Consumer Cyclical
            205: 'XLP',   # Consumer Defensive
            309: 'XLE',   # Energy
            310: 'XLI',   # Industrials
            104: 'XLRE',  # Real Estate
            101: 'XLB',   # Basic Materials
            207: 'XLU',   # Utilities
            308: 'XLC',   # Communication Services
        }
        
        if fundamentals in sector_code_map:
            return sector_code_map[fundamentals]
    except:
        pass
    
    # Default fallback for HIMS (healthcare sector)
    if ticker.upper() == 'HIMS':
        return 'XLV'
    return 'XLK'  # Default to tech


# Set sector ETF for our ticker
SECTOR_ETF = get_sector_etf(TICKER, qb) if 'qb' in dir() else 'XLV'  # HIMS is healthcare
print(f"Sector ETF for {TICKER}: {SECTOR_ETF}")

#### 6E.1 Market Regime Detection (SPY/QQQ)

In [ ]:
# ============================================================================
# MARKET REGIME DETECTION
# ============================================================================

# Run ensemble on market data
print(f"Running ensemble regime detection on {MARKET_PROXY}...")

# Fit individual models on market features
gmm_mkt = fit_gmm_regimes(market_features, n_components_range=range(3, 7))
kmeans_mkt = fit_kmeans_regimes(market_features, k_range=range(3, 7))
agg_mkt = fit_agglomerative_regimes(market_features, n_clusters=4)
cpd_mkt = detect_change_points(market_features, method='pelt', penalty=5)

# Extract labels
gmm_labels_mkt = gmm_mkt['labels']
kmeans_labels_mkt = kmeans_mkt['labels']
agg_labels_mkt = agg_mkt['labels']
cpd_labels_mkt = cpd_mkt['labels']
cpd_bkps_mkt = cpd_mkt['breakpoints']

# Ensemble classification
ensemble_labels_mkt, confidence_mkt = ensemble_regime_classification(
    gmm_labels_mkt, kmeans_labels_mkt, agg_labels_mkt, cpd_labels_mkt,
    weights=ENSEMBLE_WEIGHTS
)

# Map to meaningful labels
market_regimes = map_regime_labels(ensemble_labels_mkt, market_features, ['returns_20d', 'vol_20d'])

print(f"Market regimes identified: {market_regimes.value_counts().to_dict()}")
print(f"Average confidence: {np.mean(confidence_mkt):.2%}")

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #3a: Market Regime Over Time
# ============================================================================

def plot_regime_probabilities_enhanced(price_data: pd.DataFrame, regime_series: pd.Series,
                                       confidence_scores: np.ndarray, cpd_breakpoints: list,
                                       title: str, figsize: tuple = (14, 10)):
    """
    Create priority regime visualization with:
    - Top panel: Price chart with regime background shading
    - Middle panel: Ensemble regime classification
    - Bottom panel: Regime confidence/agreement score
    - CPD change points as vertical dashed lines
    """
    fig, axes = plt.subplots(3, 1, figsize=figsize, gridspec_kw={'height_ratios': [2, 1, 1]})
    
    # Align data
    common_idx = price_data.index.intersection(regime_series.index)
    price = price_data.loc[common_idx, 'close']
    regimes = regime_series.loc[common_idx]
    conf = pd.Series(confidence_scores, index=regime_series.index).reindex(common_idx)
    
    # ---- Panel 1: Price with regime shading ----
    ax1 = axes[0]
    ax1.plot(price.index, price.values, color='black', linewidth=1.2, label='Price')
    
    # Add regime background shading
    prev_regime = None
    segment_start = common_idx[0]
    regime_legend = set()
    
    for i, idx in enumerate(common_idx):
        current_regime = regimes.iloc[i]
        if current_regime != prev_regime and prev_regime is not None:
            color = get_regime_color(prev_regime)
            ax1.axvspan(segment_start, idx, alpha=0.3, color=color,
                       label=prev_regime if prev_regime not in regime_legend else '')
            regime_legend.add(prev_regime)
            segment_start = idx
        prev_regime = current_regime
    
    # Final segment
    if prev_regime:
        color = get_regime_color(prev_regime)
        ax1.axvspan(segment_start, common_idx[-1], alpha=0.3, color=color,
                   label=prev_regime if prev_regime not in regime_legend else '')
    
    # Add CPD change points as vertical lines
    for bp in cpd_breakpoints:
        if bp < len(common_idx):
            ax1.axvline(x=common_idx[bp], color='red', linestyle='--', linewidth=1, alpha=0.7)
    
    ax1.set_title(title, fontsize=14, fontweight='bold')
    ax1.set_ylabel('Price', fontsize=11)
    ax1.legend(loc='upper left', fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # ---- Panel 2: Regime classification ----
    ax2 = axes[1]
    
    # Convert regime names to numeric for plotting
    regime_map = {name: i for i, name in enumerate(regimes.unique())}
    regime_numeric = regimes.map(regime_map)
    
    for regime_name, regime_num in regime_map.items():
        mask = regime_numeric == regime_num
        color = get_regime_color(regime_name)
        ax2.fill_between(common_idx, 0, 1, where=mask, alpha=0.7, color=color, label=regime_name)
    
    ax2.set_ylabel('Regime', fontsize=11)
    ax2.set_yticks([])
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=9)
    
    # ---- Panel 3: Confidence score ----
    ax3 = axes[2]
    ax3.fill_between(common_idx, 0, conf.values, alpha=0.5, color='#3498db')
    ax3.plot(common_idx, conf.values, color='#2c3e50', linewidth=1)
    ax3.axhline(y=0.75, color='green', linestyle='--', label='High confidence', alpha=0.7)
    ax3.axhline(y=0.50, color='orange', linestyle='--', label='Moderate', alpha=0.7)
    
    ax3.set_ylabel('Confidence', fontsize=11)
    ax3.set_xlabel('Date', fontsize=11)
    ax3.set_ylim(0, 1.05)
    ax3.legend(loc='lower right', fontsize=9)
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


# Plot market regime visualization
plot_regime_probabilities_enhanced(
    df_market, market_regimes, confidence_mkt, cpd_bkps_mkt,
    f'PRIORITY VIZ #3a: {MARKET_PROXY} Market Regime Over Time',
    figsize=FIG_SIZE_REGIME
)

#### 6E.2 Sector Regime Detection

In [ ]:
# ============================================================================
# SECTOR REGIME DETECTION
# ============================================================================

print(f"Running ensemble regime detection on sector {SECTOR_ETF}...")

# Fit individual models on sector features
gmm_sec = fit_gmm_regimes(sector_features, n_components_range=range(3, 7))
kmeans_sec = fit_kmeans_regimes(sector_features, k_range=range(3, 7))
agg_sec = fit_agglomerative_regimes(sector_features, n_clusters=4)
cpd_sec = detect_change_points(sector_features, method='pelt', penalty=5)

# Extract labels
gmm_labels_sec = gmm_sec['labels']
kmeans_labels_sec = kmeans_sec['labels']
agg_labels_sec = agg_sec['labels']
cpd_labels_sec = cpd_sec['labels']
cpd_bkps_sec = cpd_sec['breakpoints']

# Ensemble classification
ensemble_labels_sec, confidence_sec = ensemble_regime_classification(
    gmm_labels_sec, kmeans_labels_sec, agg_labels_sec, cpd_labels_sec,
    weights=ENSEMBLE_WEIGHTS
)

# Map to meaningful labels
sector_regimes = map_regime_labels(ensemble_labels_sec, sector_features, ['returns_20d', 'vol_20d'])

print(f"Sector regimes identified: {sector_regimes.value_counts().to_dict()}")
print(f"Average confidence: {np.mean(confidence_sec):.2%}")

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #3b: Sector Regime Over Time
# ============================================================================

# Create separate plot for sector (NOT multi-panel)
plot_regime_probabilities_enhanced(
    df_sector, sector_regimes, confidence_sec, cpd_bkps_sec,
    f'PRIORITY VIZ #3b: {SECTOR_ETF} Sector Regime Over Time (with {MARKET_PROXY} overlay)',
    figsize=FIG_SIZE_REGIME
)

#### 6E.3 Asset-Specific Regime Detection

In [ ]:
# ============================================================================
# ASSET REGIME DETECTION
# ============================================================================

print(f"Running ensemble regime detection on {TICKER}...")

# Fit individual models on asset features
gmm_ast = fit_gmm_regimes(asset_features, n_components_range=range(3, 7))
kmeans_ast = fit_kmeans_regimes(asset_features, k_range=range(3, 7))
agg_ast = fit_agglomerative_regimes(asset_features, n_clusters=4)
cpd_ast = detect_change_points(asset_features, method='pelt', penalty=5)

# Extract labels
gmm_labels_ast = gmm_ast['labels']
kmeans_labels_ast = kmeans_ast['labels']
agg_labels_ast = agg_ast['labels']
cpd_labels_ast = cpd_ast['labels']
cpd_bkps_ast = cpd_ast['breakpoints']

# Ensemble classification
ensemble_labels_ast, confidence_ast = ensemble_regime_classification(
    gmm_labels_ast, kmeans_labels_ast, agg_labels_ast, cpd_labels_ast,
    weights=ENSEMBLE_WEIGHTS
)

# Map to meaningful labels focusing on:
# - Strong upward momentum
# - Moderate downward drift
# - Sideways/range-bound
# - High volatility trending
asset_regimes = map_regime_labels(ensemble_labels_ast, asset_features, ['returns_20d', 'vol_20d'])

print(f"Asset regimes identified: {asset_regimes.value_counts().to_dict()}")
print(f"Average confidence: {np.mean(confidence_ast):.2%}")

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #3c: Asset Regime Over Time
# ============================================================================

# Create separate plot for asset with market and sector overlay
plot_regime_probabilities_enhanced(
    df_asset, asset_regimes, confidence_ast, cpd_bkps_ast,
    f'PRIORITY VIZ #3c: {TICKER} Asset Regime Over Time (Multi-level Overlay)',
    figsize=FIG_SIZE_REGIME
)

In [ ]:
# ============================================================================
# REGIME STABILITY & TRANSITION ANALYSIS
# ============================================================================

def analyze_regime_transitions(regime_series: pd.Series, title: str) -> pd.DataFrame:
    """
    Calculate transition probability matrix and average duration.
    """
    transitions = calculate_regime_transitions(regime_series)
    
    # Calculate average duration in each regime
    regimes = regime_series.values
    unique_regimes = np.unique(regimes)
    durations = {r: [] for r in unique_regimes}
    
    current_regime = regimes[0]
    current_duration = 1
    
    for i in range(1, len(regimes)):
        if regimes[i] == current_regime:
            current_duration += 1
        else:
            durations[current_regime].append(current_duration)
            current_regime = regimes[i]
            current_duration = 1
    durations[current_regime].append(current_duration)
    
    avg_durations = {r: np.mean(d) if d else 0 for r, d in durations.items()}
    
    print(f"\n{title}")
    print("="*50)
    print("\nTransition Probability Matrix:")
    print(transitions.round(2))
    print("\nAverage Duration by Regime (days):")
    for regime, dur in avg_durations.items():
        print(f"  {regime}: {dur:.1f} days")
    
    return transitions


# Analyze transitions for all three levels
market_transitions = analyze_regime_transitions(market_regimes, f'{MARKET_PROXY} Market Transitions')
sector_transitions = analyze_regime_transitions(sector_regimes, f'{SECTOR_ETF} Sector Transitions')
asset_transitions = analyze_regime_transitions(asset_regimes, f'{TICKER} Asset Transitions')

In [ ]:
# ============================================================================
# CROSS-LEVEL REGIME RELATIONSHIPS
# ============================================================================

def analyze_cross_level_relationships(market_regimes: pd.Series, sector_regimes: pd.Series,
                                      asset_regimes: pd.Series):
    """
    Analyze how Market regimes influence Sector and Asset regimes.
    """
    # Align all series to common index
    common_idx = market_regimes.index.intersection(sector_regimes.index).intersection(asset_regimes.index)
    
    mkt = market_regimes.loc[common_idx]
    sec = sector_regimes.loc[common_idx]
    ast = asset_regimes.loc[common_idx]
    
    # Create contingency tables
    print("\nMARKET → SECTOR Regime Contingency:")
    print(pd.crosstab(mkt, sec, normalize='index').round(2))
    
    print("\nMARKET → ASSET Regime Contingency:")
    print(pd.crosstab(mkt, ast, normalize='index').round(2))
    
    print("\nSECTOR → ASSET Regime Contingency:")
    print(pd.crosstab(sec, ast, normalize='index').round(2))
    
    # Chi-square tests for independence
    from scipy.stats import chi2_contingency
    
    tests = [
        ('Market-Sector', pd.crosstab(mkt, sec)),
        ('Market-Asset', pd.crosstab(mkt, ast)),
        ('Sector-Asset', pd.crosstab(sec, ast)),
    ]
    
    print("\nChi-Square Independence Tests:")
    for name, table in tests:
        chi2, p_value, dof, expected = chi2_contingency(table)
        print(f"  {name}: χ² = {chi2:.1f}, p-value = {p_value:.4f} {'***' if p_value < 0.01 else '**' if p_value < 0.05 else ''}")


analyze_cross_level_relationships(market_regimes, sector_regimes, asset_regimes)

---

## Section 7: Optimal Strategy Selection & Output

This section synthesizes all analysis to determine optimal strategy allocation by regime.

### 7.1 Strategy Performance by Regime Combination

In [ ]:
# ============================================================================
# SECTION 7: OPTIMAL STRATEGY SELECTION
# ============================================================================

# Create combined regime DataFrame
common_idx = market_regimes.index.intersection(sector_regimes.index).intersection(asset_regimes.index)
common_idx = common_idx.intersection(vol_regime.dropna().index)
common_idx = common_idx.intersection(strategy_returns_df.index)

regime_df = pd.DataFrame({
    'market_regime': market_regimes.loc[common_idx],
    'sector_regime': sector_regimes.loc[common_idx],
    'asset_regime': asset_regimes.loc[common_idx],
    'vol_regime': vol_regime.loc[common_idx],
    'market_confidence': pd.Series(confidence_mkt, index=market_regimes.index).loc[common_idx],
    'sector_confidence': pd.Series(confidence_sec, index=sector_regimes.index).loc[common_idx],
    'asset_confidence': pd.Series(confidence_ast, index=asset_regimes.index).loc[common_idx]
}, index=common_idx)

# Add strategy returns
for strategy in STRATEGIES:
    regime_df[f'{strategy}_return'] = strategy_returns_df[strategy].loc[common_idx]

print(f"Combined regime data: {len(regime_df)} observations")
print(f"\nRegime combinations:")
combo_counts = regime_df.groupby(['market_regime', 'sector_regime']).size()
print(combo_counts.sort_values(ascending=False).head(10))

In [ ]:
# ============================================================================
# STRATEGY PERFORMANCE BY REGIME
# ============================================================================

def calculate_regime_performance(regime_df: pd.DataFrame, strategies: list) -> pd.DataFrame:
    """
    Calculate strategy performance metrics by regime combination.
    """
    results = []
    
    # Group by market and sector regime
    groups = regime_df.groupby(['market_regime', 'sector_regime'])
    
    for (mkt_regime, sec_regime), group in groups:
        for strategy in strategies:
            col = f'{strategy}_return'
            if col in group.columns:
                returns = group[col].dropna()
                
                if len(returns) >= 10:  # Minimum sample size
                    total_return = (1 + returns).prod() - 1
                    avg_return = returns.mean()
                    vol = returns.std()
                    sharpe = (avg_return * 252) / (vol * np.sqrt(252)) if vol > 0 else 0
                    win_rate = (returns > 0).mean()
                    
                    # Drawdown
                    cumulative = (1 + returns).cumprod()
                    running_max = cumulative.expanding().max()
                    max_dd = ((cumulative - running_max) / running_max).min()
                    
                    # Average confidence
                    avg_conf = (group['market_confidence'] + group['sector_confidence']).mean() / 2
                    
                    results.append({
                        'market_regime': mkt_regime,
                        'sector_regime': sec_regime,
                        'strategy': strategy,
                        'avg_return': avg_return,
                        'total_return': total_return,
                        'sharpe': sharpe,
                        'win_rate': win_rate,
                        'max_drawdown': max_dd,
                        'sample_size': len(returns),
                        'confidence': avg_conf
                    })
    
    return pd.DataFrame(results)


regime_performance = calculate_regime_performance(regime_df, STRATEGIES)

print("Strategy Performance by Regime:")
print(regime_performance.sort_values('sharpe', ascending=False).head(20).to_string(index=False))

### 7.2 PRIORITY VISUALIZATION #4: Strategy Performance Heatmaps by Regime

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #4: Performance Heatmaps by Strategy
# ============================================================================

def plot_regime_performance_heatmap(perf_df: pd.DataFrame, strategy: str):
    """
    Create heatmap showing strategy performance across regime combinations.
    """
    strategy_data = perf_df[perf_df['strategy'] == strategy]
    
    if strategy_data.empty:
        print(f"No data for {strategy}")
        return
    
    # Pivot for heatmap
    pivot_return = strategy_data.pivot(index='market_regime', columns='sector_regime', 
                                        values='avg_return').fillna(0)
    pivot_winrate = strategy_data.pivot(index='market_regime', columns='sector_regime',
                                         values='win_rate').fillna(0)
    pivot_sample = strategy_data.pivot(index='market_regime', columns='sector_regime',
                                        values='sample_size').fillna(0)
    
    fig, ax = plt.subplots(figsize=FIG_SIZE_HEATMAP)
    
    # Main heatmap with returns
    color = get_strategy_color(strategy)
    
    # Use diverging colormap
    vmax = max(abs(pivot_return.values.min()), abs(pivot_return.values.max()))
    vmin = -vmax
    
    sns.heatmap(pivot_return * 100, annot=True, fmt='.2f', cmap='RdYlGn',
                ax=ax, center=0, vmin=vmin*100, vmax=vmax*100,
                linewidths=0.5, cbar_kws={'label': 'Avg Daily Return (%)'})
    
    # Add sample size annotations
    for i in range(len(pivot_return.index)):
        for j in range(len(pivot_return.columns)):
            n = pivot_sample.iloc[i, j]
            wr = pivot_winrate.iloc[i, j]
            if n > 0:
                ax.annotate(f'n={int(n)}\nWR={wr:.0%}',
                           xy=(j + 0.5, i + 0.7),
                           ha='center', va='center',
                           fontsize=8, alpha=0.7)
    
    ax.set_title(f'{strategy} - Performance by Regime Combination', 
                 fontsize=14, color=color, fontweight='bold')
    ax.set_xlabel('Sector Regime')
    ax.set_ylabel('Market Regime')
    
    plt.tight_layout()
    plt.show()


# Create separate heatmaps for each strategy
for strategy in STRATEGIES:
    plot_regime_performance_heatmap(regime_performance, strategy)
    print()

### 7.3 Optimal Strategy Selection by Regime

In [ ]:
# ============================================================================
# OPTIMAL STRATEGY BY REGIME
# ============================================================================

def find_optimal_strategy(perf_df: pd.DataFrame) -> pd.DataFrame:
    """
    Find the optimal strategy for each regime combination.
    """
    # For each regime combination, find best strategy by Sharpe
    optimal = []
    
    groups = perf_df.groupby(['market_regime', 'sector_regime'])
    
    for (mkt, sec), group in groups:
        if len(group) > 0:
            best = group.loc[group['sharpe'].idxmax()]
            
            # Also find optimal stop loss (simplified - use best overall for now)
            stop_data = stop_loss_df[stop_loss_df['strategy'] == best['strategy']]
            if len(stop_data) > 0:
                best_stop = stop_data.loc[stop_data['sharpe'].idxmax(), 'stop_loss']
            else:
                best_stop = 'None'
            
            optimal.append({
                'Regime_Market': mkt,
                'Regime_Sector': sec,
                'Recommended_Strategy': best['strategy'],
                'Expected_Return': best['avg_return'],
                'Sharpe': best['sharpe'],
                'Max_DD': best['max_drawdown'],
                'Win_Rate': best['win_rate'],
                'Sample_Size': best['sample_size'],
                'Ensemble_Confidence': best['confidence'],
                'Stop_Loss_%': best_stop
            })
    
    return pd.DataFrame(optimal)


optimal_strategies = find_optimal_strategy(regime_performance)

print("OPTIMAL STRATEGY ALLOCATION BY REGIME:")
print("="*80)
print(optimal_strategies.to_string(index=False))

In [ ]:
# ============================================================================
# COMPREHENSIVE OUTPUT TABLE
# ============================================================================

# Add volatility regime and asset regime for complete picture
def create_comprehensive_recommendations() -> pd.DataFrame:
    """
    Create comprehensive strategy recommendations table.
    """
    # Get unique regime combinations
    combos = regime_df.groupby(['market_regime', 'sector_regime', 'asset_regime', 'vol_regime']).size()
    combos = combos[combos >= 5].reset_index(name='sample_size')  # Min 5 days
    
    recommendations = []
    
    for _, row in combos.iterrows():
        mkt, sec, ast, vol = row['market_regime'], row['sector_regime'], row['asset_regime'], row['vol_regime']
        
        # Filter data for this regime combination
        mask = (
            (regime_df['market_regime'] == mkt) &
            (regime_df['sector_regime'] == sec) &
            (regime_df['asset_regime'] == ast) &
            (regime_df['vol_regime'] == vol)
        )
        subset = regime_df[mask]
        
        if len(subset) < 5:
            continue
        
        # Find best strategy
        best_strategy = None
        best_sharpe = -np.inf
        best_return = 0
        best_winrate = 0
        
        for strategy in STRATEGIES:
            col = f'{strategy}_return'
            if col in subset.columns:
                returns = subset[col].dropna()
                if len(returns) >= 5:
                    avg_ret = returns.mean()
                    vol_ret = returns.std()
                    sharpe = (avg_ret * 252) / (vol_ret * np.sqrt(252)) if vol_ret > 0 else 0
                    win_rate = (returns > 0).mean()
                    
                    if sharpe > best_sharpe:
                        best_sharpe = sharpe
                        best_strategy = strategy
                        best_return = avg_ret
                        best_winrate = win_rate
        
        if best_strategy:
            # Determine delta based on strategy type
            call_delta = 'N/A'
            put_delta = 'N/A'
            
            if 'Call' in best_strategy:
                call_delta = '0.40'
            if 'Put' in best_strategy:
                put_delta = '-0.20'
            if best_strategy in ['Straddle', 'Strangle']:
                call_delta = '0.30'
                put_delta = '-0.30'
            if 'Calendar' in best_strategy or 'Diagonal' in best_strategy:
                call_delta = '0.50'
            
            # Get recommended stop loss
            stop_data = stop_loss_df[stop_loss_df['strategy'] == best_strategy]
            if len(stop_data) > 0:
                stop_loss = stop_data.loc[stop_data['sharpe'].idxmax(), 'stop_loss']
            else:
                stop_loss = 'None'
            
            # Average confidence
            avg_conf = subset[['market_confidence', 'sector_confidence', 'asset_confidence']].mean().mean()
            
            recommendations.append({
                'Regime_Market': mkt,
                'Regime_Sector': sec,
                'Regime_Asset': ast,
                'Volatility_Regime': vol,
                'Ensemble_Confidence': f'{avg_conf:.2f}',
                'Recommended_Strategy': best_strategy,
                'Call_Delta': call_delta,
                'Put_Delta': put_delta,
                'Stop_Loss_%': stop_loss,
                'Expected_Return': f'{best_return*100:.2f}%',
                'Sharpe': f'{best_sharpe:.2f}',
                'Win_Rate': f'{best_winrate:.0%}',
                'Sample_Size': len(subset)
            })
    
    return pd.DataFrame(recommendations)


comprehensive_recommendations = create_comprehensive_recommendations()

print("\nCOMPREHENSIVE STRATEGY RECOMMENDATIONS:")
print("="*120)
print(comprehensive_recommendations.to_string(index=False))

# Save to CSV
# comprehensive_recommendations.to_csv('strategy_recommendations.csv', index=False)
# print("\nRecommendations saved to strategy_recommendations.csv")

### 7.4 Current Regime Analysis

In [ ]:
# ============================================================================
# CURRENT REGIME ANALYSIS
# ============================================================================

# Get current (most recent) regime states
current_market = market_regimes.iloc[-1]
current_sector = sector_regimes.iloc[-1]
current_asset = asset_regimes.iloc[-1]
current_vol = vol_regime.iloc[-1] if len(vol_regime) > 0 else 'Unknown'

current_market_conf = confidence_mkt[-1] if len(confidence_mkt) > 0 else 0
current_sector_conf = confidence_sec[-1] if len(confidence_sec) > 0 else 0
current_asset_conf = confidence_ast[-1] if len(confidence_ast) > 0 else 0

print("CURRENT REGIME STATES:")
print("="*60)
print(f"Date: {regime_df.index[-1]}")
print(f"\nMarket Regime: {current_market} (Confidence: {current_market_conf:.2%})")
print(f"Sector Regime: {current_sector} (Confidence: {current_sector_conf:.2%})")
print(f"Asset Regime:  {current_asset} (Confidence: {current_asset_conf:.2%})")
print(f"Volatility:    {current_vol}")

# Find recommended strategy for current regime
current_rec = comprehensive_recommendations[
    (comprehensive_recommendations['Regime_Market'] == current_market) &
    (comprehensive_recommendations['Regime_Sector'] == current_sector)
]

if len(current_rec) > 0:
    print("\nRECOMMENDED STRATEGY FOR CURRENT REGIME:")
    print("-"*60)
    rec = current_rec.iloc[0]
    print(f"Strategy: {rec['Recommended_Strategy']}")
    print(f"Call Delta: {rec['Call_Delta']}")
    print(f"Put Delta: {rec['Put_Delta']}")
    print(f"Stop Loss: {rec['Stop_Loss_%']}")
    print(f"Expected Return: {rec['Expected_Return']}")
    print(f"Sharpe Ratio: {rec['Sharpe']}")
    print(f"Win Rate: {rec['Win_Rate']}")
else:
    print("\nNo historical data for current regime combination.")
    print("Consider using the most similar regime or conservative positioning.")

### 7.5 PRIORITY VISUALIZATION #5: Summary Dashboard

In [ ]:
# ============================================================================
# PRIORITY VISUALIZATION #5: Summary Dashboard
# ============================================================================

fig = plt.figure(figsize=FIG_SIZE_DASHBOARD)

# Create 3x3 grid
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Panel 1: Current regime states
ax1 = fig.add_subplot(gs[0, 0])
regimes = ['Market', 'Sector', 'Asset']
regime_values = [current_market, current_sector, current_asset]
confidence_values = [current_market_conf, current_sector_conf, current_asset_conf]
colors = [get_regime_color(r) for r in regime_values]

bars = ax1.barh(regimes, confidence_values, color=colors, alpha=0.7)
ax1.set_xlim(0, 1)
ax1.set_xlabel('Confidence')
ax1.set_title('Current Regime States', fontsize=12)

for bar, val, regime in zip(bars, confidence_values, regime_values):
    ax1.annotate(f'{regime}\n{val:.0%}', 
                xy=(val + 0.02, bar.get_y() + bar.get_height()/2),
                va='center', fontsize=9)

# Panel 2: Volatility distribution
ax2 = fig.add_subplot(gs[0, 1])
vol_hist = realized_vol['rv_true'].dropna()
current_vol_value = vol_hist.iloc[-1] if len(vol_hist) > 0 else 0

ax2.hist(vol_hist, bins=30, alpha=0.7, color='#3498db', edgecolor='black')
ax2.axvline(x=current_vol_value, color='red', linewidth=2, label=f'Current: {current_vol_value:.2%}')
ax2.axvline(x=vol_hist.mean(), color='green', linestyle='--', label=f'Mean: {vol_hist.mean():.2%}')
ax2.set_title('Volatility Distribution', fontsize=12)
ax2.set_xlabel('Volatility')
ax2.legend(fontsize=8)

# Panel 3: Recommended strategy
ax3 = fig.add_subplot(gs[0, 2])
ax3.axis('off')

if len(current_rec) > 0:
    rec = current_rec.iloc[0]
    strategy_color = get_strategy_color(rec['Recommended_Strategy'])
    
    text = f"RECOMMENDED STRATEGY\n\n"
    text += f"{rec['Recommended_Strategy']}\n\n"
    text += f"Stop Loss: {rec['Stop_Loss_%']}\n"
    text += f"Expected Return: {rec['Expected_Return']}\n"
    text += f"Sharpe: {rec['Sharpe']}"
    
    ax3.text(0.5, 0.5, text, ha='center', va='center', fontsize=11,
             transform=ax3.transAxes,
             bbox=dict(boxstyle='round', facecolor=strategy_color, alpha=0.3))
else:
    ax3.text(0.5, 0.5, 'No recommendation\nfor current regime', 
             ha='center', va='center', fontsize=11, transform=ax3.transAxes)

ax3.set_title('Strategy Recommendation', fontsize=12)

# Panel 4: Performance metrics table
ax4 = fig.add_subplot(gs[1, 0])
ax4.axis('off')

if len(current_rec) > 0:
    rec = current_rec.iloc[0]
    table_data = [
        ['Metric', 'Value'],
        ['Win Rate', rec['Win_Rate']],
        ['Sharpe', rec['Sharpe']],
        ['Sample Size', str(rec['Sample_Size'])],
        ['Confidence', rec['Ensemble_Confidence']]
    ]
    
    table = ax4.table(cellText=table_data, loc='center', cellLoc='center',
                      colWidths=[0.4, 0.4])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)

ax4.set_title('Expected Performance', fontsize=12)

# Panel 5: Historical performance of recommended strategy
ax5 = fig.add_subplot(gs[1, 1])

if len(current_rec) > 0:
    rec_strategy = current_rec.iloc[0]['Recommended_Strategy']
    strategy_returns_series = strategy_returns_df[rec_strategy]
    cumulative = (1 + strategy_returns_series).cumprod()
    
    color = get_strategy_color(rec_strategy)
    ax5.plot(cumulative, color=color, linewidth=1.5)
    ax5.set_title(f'{rec_strategy} Historical Performance', fontsize=12)
    ax5.set_ylabel('Cumulative Return')
    ax5.grid(True, alpha=0.3)

# Panel 6: Risk warnings
ax6 = fig.add_subplot(gs[1, 2])
ax6.axis('off')

# Check for warnings
warnings = []
avg_conf = (current_market_conf + current_sector_conf + current_asset_conf) / 3

if avg_conf < 0.6:
    warnings.append("⚠️ Low regime confidence")
if current_vol == 'Extreme':
    warnings.append("⚠️ Extreme volatility detected")
if len(current_rec) == 0:
    warnings.append("⚠️ Novel regime combination")
if len(current_rec) > 0 and int(current_rec.iloc[0]['Sample_Size']) < 20:
    warnings.append("⚠️ Limited historical data")

if warnings:
    warning_text = "RISK WARNINGS\n\n" + "\n".join(warnings)
    ax6.text(0.5, 0.5, warning_text, ha='center', va='center', fontsize=10,
             transform=ax6.transAxes,
             bbox=dict(boxstyle='round', facecolor='#ffcccc', alpha=0.5))
else:
    ax6.text(0.5, 0.5, '✅ No significant warnings\n\nRegime stable with\nhigh confidence',
             ha='center', va='center', fontsize=10, transform=ax6.transAxes,
             bbox=dict(boxstyle='round', facecolor='#ccffcc', alpha=0.5))

ax6.set_title('Risk Alerts', fontsize=12)

# Panel 7: Method agreement pie chart
ax7 = fig.add_subplot(gs[2, 0])

# Calculate method agreement
agreement_levels = {
    'High (75%+)': (confidence_mkt >= 0.75).mean(),
    'Moderate (50-75%)': ((confidence_mkt >= 0.5) & (confidence_mkt < 0.75)).mean(),
    'Low (<50%)': (confidence_mkt < 0.5).mean()
}

colors_pie = ['#2ecc71', '#f39c12', '#e74c3c']
ax7.pie([v for v in agreement_levels.values()], labels=agreement_levels.keys(),
        autopct='%1.0f%%', colors=colors_pie, startangle=90)
ax7.set_title('Ensemble Agreement Distribution', fontsize=12)

# Panel 8: PCA feature space (simplified 2D)
ax8 = fig.add_subplot(gs[2, 1])

# Quick PCA for visualization
scaler_viz = StandardScaler()
X_viz = scaler_viz.fit_transform(asset_features.dropna())
pca_viz = PCA(n_components=2)
X_pca = pca_viz.fit_transform(X_viz)

# Color by regime
colors_scatter = [get_regime_color(r) for r in asset_regimes.loc[asset_features.dropna().index]]
ax8.scatter(X_pca[:, 0], X_pca[:, 1], c=colors_scatter, alpha=0.5, s=10)
ax8.scatter(X_pca[-1, 0], X_pca[-1, 1], c='black', s=100, marker='*', 
            label='Current', edgecolors='white', linewidth=1)
ax8.set_xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.0%})')
ax8.set_ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.0%})')
ax8.set_title('Feature Space Position', fontsize=12)
ax8.legend(loc='upper right', fontsize=9)
ax8.grid(True, alpha=0.3)

# Panel 9: Regime transition probabilities
ax9 = fig.add_subplot(gs[2, 2])

# Get transition probabilities from current regime
if current_vol in transition_probs.index:
    trans = transition_probs.loc[current_vol]
    colors_bar = [get_vol_regime_color(r) for r in trans.index]
    ax9.bar(trans.index, trans.values, color=colors_bar, alpha=0.7)
    ax9.set_ylim(0, 1)
    ax9.set_ylabel('Probability')
    ax9.set_title(f'Next Regime Probability\n(from {current_vol})', fontsize=12)
    ax9.grid(True, alpha=0.3, axis='y')
else:
    ax9.text(0.5, 0.5, 'Transition data\nnot available', 
             ha='center', va='center', transform=ax9.transAxes)
    ax9.set_title('Regime Transition', fontsize=12)

plt.suptitle(f'{TICKER} - Volatility Trading Dashboard', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

### 7.6 Portfolio Extension Framework

In [ ]:
# ============================================================================
# PORTFOLIO EXTENSION FRAMEWORK
# ============================================================================

# This section provides structure for future multi-asset portfolio optimization

class VolatilityTradingPortfolio:
    """
    Framework for multi-asset volatility trading portfolio.
    
    Designed to be extended for portfolio-level optimization.
    """
    
    def __init__(self, tickers: List[str]):
        self.tickers = tickers
        self.asset_data = {}
        self.regime_data = {}
        self.strategy_allocations = {}
        
    def add_asset(self, ticker: str, data: pd.DataFrame, regimes: pd.Series):
        """Add asset data to portfolio."""
        self.asset_data[ticker] = data
        self.regime_data[ticker] = regimes
        
    def calculate_cross_asset_correlation(self) -> pd.DataFrame:
        """Calculate correlation matrix across assets."""
        returns_df = pd.DataFrame()
        for ticker, data in self.asset_data.items():
            returns_df[ticker] = data['returns'] if 'returns' in data.columns else data['close'].pct_change()
        return returns_df.corr()
    
    def calculate_portfolio_greeks(self, positions: dict) -> dict:
        """
        Calculate aggregate portfolio Greeks.
        
        Parameters:
        -----------
        positions : dict
            Dictionary of position sizes by ticker and strategy
            
        Returns:
        --------
        dict
            Aggregate delta, gamma, theta, vega
        """
        # Placeholder for portfolio-level Greek calculations
        return {
            'portfolio_delta': 0,
            'portfolio_gamma': 0,
            'portfolio_theta': 0,
            'portfolio_vega': 0
        }
    
    def check_regime_correlation(self) -> pd.DataFrame:
        """Check if assets share regimes simultaneously."""
        regime_df = pd.DataFrame()
        for ticker, regimes in self.regime_data.items():
            regime_df[ticker] = pd.Categorical(regimes).codes
        return regime_df.corr()
    
    def optimize_allocation(self, constraints: dict = None) -> dict:
        """
        Optimize strategy allocation across assets.
        
        Placeholder for future implementation of:
        - Mean-variance optimization
        - Risk parity
        - Regime-based allocation
        """
        # Placeholder - to be implemented
        return {ticker: {'strategy': None, 'weight': 0} for ticker in self.tickers}


# Example usage (for future expansion)
print("Portfolio Extension Framework initialized.")
print("")
print("Key methods for future implementation:")
print("  - add_asset(): Add multiple assets to portfolio")
print("  - calculate_cross_asset_correlation(): Diversification analysis")
print("  - calculate_portfolio_greeks(): Aggregate risk exposure")
print("  - check_regime_correlation(): Regime synchronization")
print("  - optimize_allocation(): Multi-asset strategy allocation")

---

## Summary & Conclusions

This notebook has implemented a comprehensive volatility trading research framework including:

### Key Components:
1. **Volatility Modeling**: Multiple models (Historical, EWMA, Parkinson, Garman-Klass, Rogers-Satchell, ATR) with PCA-based aggregate forecasting

2. **Mean Reversion Analysis**: Volatility regime classification with transition probabilities and DTE recommendations

3. **Options Strategies**: Six strategies (Calendar Spread, Double Diagonal, Straddle, Strangle, Bull Put Spread, Bull Call Spread) with delta optimization

4. **Ensemble Regime Detection**: Combined GMM, K-Means, Agglomerative Clustering, and Change-Point Detection for robust regime identification at Market, Sector, and Asset levels

5. **Stop Loss Optimization**: Systematic testing of stop loss thresholds by strategy

6. **Comprehensive Output**: Machine-readable recommendations table with regime-specific strategy allocation

### Technical Implementation:
- All computations use vectorized operations (NumPy/Pandas)
- No multiprocessing (QuantConnect compatible)
- Ticker-agnostic design for easy portfolio extension
- Consistent color palettes across all visualizations

### Next Steps:
1. Connect to live QuantConnect options data
2. Implement actual options pricing models
3. Extend to multi-asset portfolio optimization
4. Add real-time regime monitoring
5. Backtest with actual transaction costs

In [ ]:
# ============================================================================
# END OF NOTEBOOK
# ============================================================================

print("="*60)
print("VOLATILITY TRADING RESEARCH NOTEBOOK COMPLETE")
print("="*60)
print(f"")
print(f"Ticker analyzed: {TICKER}")
print(f"Date range: {START_DATE.date()} to {END_DATE.date()}")
print(f"")
print(f"Current Regimes:")
print(f"  Market: {current_market}")
print(f"  Sector: {current_sector}")
print(f"  Asset:  {current_asset}")
print(f"  Volatility: {current_vol}")
print(f"")

if len(current_rec) > 0:
    rec = current_rec.iloc[0]
    print(f"Recommended Strategy: {rec['Recommended_Strategy']}")
    print(f"Stop Loss: {rec['Stop_Loss_%']}")
else:
    print("No specific recommendation for current regime.")

print(f"")
print("See comprehensive_recommendations DataFrame for full regime-strategy mapping.")